In [1]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch
from basic.claims import connect #, get_claims, get_users, get_kibana_gate, products_bankiros, nulls, get_medium, get_claims_raw
# from basic.external import cpa_parse, get_yandex_data
# from basic.dscore import cre_get, mfo_score
from basic.reports import cre_keyword_report, master_report, registrations_medium
from basic.vars import regions_mapper, claim_offers
from bson.objectid import ObjectId
import requests
import json
from datetime import datetime, date, timedelta
import pandas as pd
from time import sleep
from base64 import b64encode
import numpy as np
# from robobrowser import RoboBrowser
import re
# import csv, sqlite3
# import sqlalchemy
import matplotlib.pyplot as plt
import seaborn as sns
from calendar import monthrange
import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth',1000)
# from IPython.display import Filelink, FileLinks


client, es = connect()

/opt/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Доспуп к Эластику 	 True
Доспуп к Монге 		 True


In [2]:
def get_registrations(userIds):
    userIds = list(set(userIds))
    dp = pd.DataFrame()
    for i in range(int(len(userIds)/1000 + 1)):
        payload = {"size": 5000,
                   "query":{"bool":{"must":
                  [{"match_all":{}},
                  {"bool":{"should": [{"match_phrase":{"params.userId.keyword": _id }} for 
                                      _id in userIds[i*1000:(i+1)*1000]],
                                      "minimum_should_match":1 }},
                                     {"match_phrase":{"eventType.keyword":{"query":"REGISTRATION"}}}]}}}
        d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=60)
        print(d['hits']['total'], end="|")
        pt = {"card": "Кредитная карта",
              "Кредитная карта":"Кредитная карта",
              "cash": "Кредит наличными",
              "Кредит наличными": "Кредит наличными",
              "loan": "Займ",
              "Займ": "Займ",
              "refinance": "Рефинансирование",
              "Рефинансирование": "Рефинансирование",}

        dp1 = pd.DataFrame(d['hits']['hits'])
        dp = pd.concat([dp, dp1])
    dp['userId'] = dp['_source'].apply(lambda x: x['params']['userId'])
    dp['transaction_id'] = dp['_source'].apply(lambda x: x['params']['transaction_id'] if 
                        'transaction_id' in x['params'].keys() else np.nan)
    dp['productType'] = dp['_source'].apply(lambda x: x['params']['productType'] if 
                        'productType' in x['params'].keys() else np.nan)
    dp['productType'] = dp['productType'].map(pt)
    dp['utm_source'] = dp['_source'].apply(lambda x: x['params']['utm_source'] if 
                        'utm_source' in x['params'].keys() else np.nan)
    dp['utm_medium'] = dp['_source'].apply(lambda x: x['params']['utm_medium'] if 
                        'utm_medium' in x['params'].keys() else np.nan)
    dp['reg_time'] = dp['_source'].apply(lambda x: x['timestamp'] if 
                        'timestamp' in x.keys() else np.nan)
    dp['reg_time'] = pd.to_datetime(dp['reg_time']).apply(lambda x: x.replace(tzinfo=None))
#     print(dp['_source'].tolist())
    dp = dp[['userId', 'transaction_id', 'productType', 'utm_source', 'utm_medium', 'reg_time']]
    return dp

In [3]:
#!pip install robobrowser

In [4]:
# !pip install pickledb pymongo elasticsearch==6.8 oauth2client gspread df2gspread \
# robobrowser beautifulsoup4 lxml yourls Werkzeug==0.16.1 paramiko xlsxwriter pyminizip \
# html5lib zodiac_sign python-pptx plotly df2gspread gspread

In [5]:
#!pip install gspread_dataframe

In [6]:
def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)    
            
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)  # сюда json файл если он не в той же папке
            creds = flow.run_local_server(port=0)        
            
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)    
    return creds

def pull_sheet_data(SCOPES,SPREADSHEET_ID,RANGE_NAME):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE_NAME).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=RANGE_NAME).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data
    
def to_float(x):
    try:
        return(float(x.replace("\xa0", '').replace(" ", '').strip()))
    except Exception as e:
        return np.nan

In [7]:
def cre_get(es, since):
    
    chunk = 10000
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    payload = {"size": chunk,
               "_source": ["params.parsed"],
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword": "CRE"}},
                         {"range":{"timestamp":{"gte":gte,"format":"epoch_millis"}}}
                        ]}},
"_source": ["params.parsed", "timestamp"]
              }
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=3000)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/chunk)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=3000)
        hits = hits + d['hits']['hits']
        print(doc_count-chunk*i, end ="; ", flush=True)
        sleep(0.5)
#         break
    print(len(hits), 'raz')
    cre = []
    for h in hits:
        if len(h['_source']['params']['parsed'])>10:
            h2 = json.loads(h['_source']['params']['parsed'].replace("'", '"'))
            h2.update({"timestamp": pd.to_datetime(h['_source']['timestamp'])})
            cre.append(h2)
    df = pd.DataFrame(cre).sort_values("timestamp", 
                                       ascending=False).drop_duplicates('userId', keep='first').reset_index(drop=True)
    df.index = df['userId']
      
            
    return df['score'].to_dict()

In [8]:
def mm_requestId(x):
    if type(x)==dict and 'NOTE' in x.keys():
        if x['NOTE'].find('HTTP_RESPONSE_BODY') >0:
            htr = json.loads(x['NOTE'][x['NOTE'].find('HTTP_RESPONSE_BODY')+20:])
            htr = htr['requestId'] if 'requestId' in htr.keys() else np.nan
#             print(htr)
        else: 
            htr = np.nan
    else: 
        htr = np.nan
    return htr

In [9]:
def get_exact_claims(since, till, client):
    req = {"createdAt": {"$gte": datetime.combine(since, datetime.min.time()) - timedelta(hours=3),
                          "$lt": datetime.combine(till, datetime.min.time())- timedelta(hours=3)},
#            "product._id": {"$in": [ObjectId("5e9846be79dd27000141f476"),
# ObjectId("5ef4473acf22a90001010445")]},
           #"product._id": {"$in": [ObjectId(product) for product in products]}}
            "status": {"$nin": [ "ERROR", "NEW1"]}} 
    results_count = client['claim-service']['claims'].count_documents(req)
    # print(req)
    print(results_count)
    all_ans = []
    batch_size = 10000
    for i in range(results_count//batch_size+1):
        ans = list(client['claim-service']['claims'].find(req, {"product": 1, "partner":1,
                                                                "borrower": 1, "statusFields":1,
                                                                "status": 1, "adapterName": 1,
                                                                "createdAt": 1, 'type':1,
                                                                "updatedAt":1, "clickId": 1,
                                                                "whence":1}).limit(batch_size).skip(i*batch_size))
        all_ans = all_ans + ans
    #ans = trans(all_ans, client)
    df = pd.DataFrame(all_ans)
    df['id'] = df["_id"].apply(lambda x: str(x))
    df['dt'] = df["createdAt"].apply(lambda x: (x+timedelta(hours=3)).date())
    df['productId'] = df['product'].apply(lambda x: str(x['_id']))
    df['product_type'] = df['product'].apply(lambda x: x['productType'] if 'productType' in x.keys() else np.nan)
    df['product_name'] = df['product'].apply(lambda x: x['name'])
    df['partner'] = df['partner'].apply(lambda x: x['name'])
    df['userId'] = df['borrower'].apply(lambda x: x['userId'])
    df['phone'] = df['borrower'].apply(lambda x: x['phone'] if 'phone' in x.keys() else np.nan)

    return df


def get_kibana_claims(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 10000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"CLAIM"}},
#                          {"match_phrase":{"params.sourceGlobal.keyword":{"query": sourceGlobal }}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/10000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-10000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
                  'transaction_id', 'clickId', 'sourceGlobal', 'utm_content','orderId','transactionId']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['date'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

In [10]:
def get_kibana_postback(es, since):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(date.today()+timedelta(days=1), datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 10000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"POSTBACK"}},
#                          {"match_phrase":{"params.whence.keyword":{"query": whence }}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/10000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-10000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', 'payout', 'status',
                'transaction_id', 'externalId','orderId','transactionId']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['date'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

In [11]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000
# filters = "ym:s:UTMSource=='smsTopBP'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'utm_source':result[i]["dimensions"][1]["name"],
#             'visit-id':result[i]["dimensions"][3]["name"],
            #'area':result[i]["dimensions"][4]["name"],
#             'users':result[i]["metrics"][0],
             'visits':result[i]["metrics"][0],
            #'utm_term':result[i]["dimensions"][2]["name"],
#'avgVisitDurationSeconds':result[i]["dimensions"][3]["name"],
           #'yandexClientId':result[i]["dimensions"][5]["name"]
            #'startURL':result[i]["dimensions"][7]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] = (df['date'].dt.week)+1
df = df.groupby('week').agg({'visits':'sum'})
df = df.T
df = df.sort_index(axis='columns', level='week',ascending=False)
df.to_excel('Недельный отчет/trafic_new_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(0) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/trafic_new_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
df

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


,Unnamed: 0,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2
0,visits,26700,14229,13388,13138,15512,17560,13135,13521,17948,12520,20017,14016,9762,14175,8982,8128,8733,8812,13893,3599,7571,8302,16449,13297,10688,12763,16137,12019,8948,9141,13675,23343,20699,22287,22997,29284,20754,15506,20275,19388,22845,23319,23901,22498,23627,24079,24609


In [12]:
#!pip install openpyxl

In [13]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000
# filters = "ym:s:UTMSource=='smsTopBP'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
            #'visit-id':result[i]["dimensions"][2]["name"],
            #'utm_medium':result[i]["dimensions"][2]["name"],
       #'utm_campaign':result[i]["dimensions"][2]["name"],
        #'utm_content':result[i]["dimensions"][4]["name"],
        #'utm_term':result[i]["dimensions"][5]["name"],
#             'users':result[i]["metrics"][0],
             'visits':result[i]["metrics"][0],
            #'utm_term':result[i]["dimensions"][2]["name"],
#'avgVisitDurationSeconds':result[i]["dimensions"][3]["name"],
           #'yandexClientId':result[i]["dimensions"][5]["name"]
            #'startURL':result[i]["dimensions"][7]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] = (df['date'].dt.week)+1

# df.to_excel('trafic_new_Alexander.xlsx')
# import pandas as pd
# import gspread
# from gspread_dataframe import set_with_dataframe

# # Доступ к GS
# gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
# sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
# worksheet = sh.get_worksheet(0) #-> 0 - лист один, 1 - второй и тд.

# # Выбор датафрейма
# df = pd.read_excel('trafic_new_Alexander.xlsx'
#                   )
# set_with_dataframe(worksheet, df) 
df 

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:59: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


,date,utm_source,visits,week
0,2021-01-04,None,2826.0,2
1,2021-01-04,admitad,82.0,2
2,2021-01-04,advcake,159.0,2
3,2021-01-04,ais_google,141.0,2
4,2021-01-04,ais_yandex,271.0,2
...,...,...,...,...
7413,2021-11-29,leadgid2,5.0,49
7414,2021-11-29,sms_trigger,7.0,49
7415,2021-11-29,smsmonthly,2.0,49
7416,2021-11-29,tele2,197.0,49


In [14]:
df_for_none = df

In [15]:
df_none=df_for_none[df_for_none['utm_source'].isnull()].groupby('week',as_index=False).agg({'visits':'sum'})
df_none

,week,visits
0,2,18605.0
1,3,20901.0
2,4,21150.0
3,5,19951.0
4,6,20821.0
5,7,20419.0
6,8,20041.0
7,9,16653.0
8,10,18040.0
9,11,13865.0


In [16]:
#df['utm_source']=df['utm_source'].fillna('Не определено')

In [17]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:searchEngine",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000
    #filters = "ym:s:lastTrafficSource=='organic'"
# filters = "ym:s:UTMSource=='smsTopBP'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
#             'visit-id':result[i]["dimensions"][3]["name"],
            #'area':result[i]["dimensions"][1]["name"],
#             'users':result[i]["metrics"][0],
             'visits':result[i]["metrics"][0],
            #'gender':result[i]["dimensions"][2]["name"],
#'avgVisitDurationSeconds':result[i]["dimensions"][3]["name"],
           #'yandexClientId':result[i]["dimensions"][5]["name"]
            #'startURL':result[i]["dimensions"][7]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df_seo = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df_seo['date'] = pd.to_datetime(df_seo['date'])
df_seo['week'] =(df_seo['date'].dt.week)+1
df_seo = df_seo.groupby(['week','utm_source'],as_index=False).agg({'visits':'sum'})
df_seo = df_seo.groupby(['week'],as_index=False).agg({'visits':'sum'})

df_seo['Отдел']='SEO'
df_seo

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:57: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


,week,visits,Отдел
0,2,2130.0,SEO
1,3,2423.0,SEO
2,4,2369.0,SEO
3,5,1945.0,SEO
4,6,1770.0,SEO
5,7,1864.0,SEO
6,8,1965.0,SEO
7,9,1879.0,SEO
8,10,2148.0,SEO
9,11,1424.0,SEO


In [18]:
df_none1 = df_none.merge(df_seo,how='left',on='week')
df_none1['visits'] = df_none1['visits_x']- df_none1['visits_y']
df_none2= df_none1[['week','visits']]
df_none2['utm_source']='Прочее'
df_none2

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,week,visits,utm_source
0,2,16475.0,Прочее
1,3,18478.0,Прочее
2,4,18781.0,Прочее
3,5,18006.0,Прочее
4,6,19051.0,Прочее
5,7,18555.0,Прочее
6,8,18076.0,Прочее
7,9,14774.0,Прочее
8,10,15892.0,Прочее
9,11,12441.0,Прочее


In [19]:
df=df[~df['utm_source'].isnull()]
df = df.groupby(['week','utm_source'],as_index=False).agg({'visits':'sum'})
df= pd.concat([df,df_none2])
df

,week,utm_source,visits
0,2,admitad,881.0
1,2,advcake,827.0
2,2,advertise,1.0
3,2,ais_google,1044.0
4,2,ais_yandex,2551.0
...,...,...,...
43,45,Прочее,4707.0
44,46,Прочее,4806.0
45,47,Прочее,5193.0
46,48,Прочее,6449.0


In [20]:
df['utm_source'] = df['utm_source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Vbr.ru ссылки' if 
                                  x in ['vbrru', 'upside_vbr','vbrr','vbr'] else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Email' if 
                                 x[:5]=='email' or x[:7]=='TRIGGER' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'SMS рекламные' if 
                                   x[:3]=='sms' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'SMS рекламные' if 
                                   x in ['tele2TommorowWON',
                                         'tele2Tommorow',
                                         'tele2FourDays',
                                         'tele2TOP'] or x[:5]=='t2sms' or x =='lostBasket' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Admitad' if 
                                   x[:7]=='admitad' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Adv.Cake' if 
                                   x[:7]=='advcake' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Leadgid' if 
                                   x[:7]=='leadgid' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Rubid' if 
                                   x[:5]=='rubid' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Cpahub' if 
                                   x[:6]=='cpahub' or x[:6]=='Cpahub' or x== 'get4click'
                                          or x =='checkscan' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Перфоманс через Артикс' if 
                                   x[:3]=='ais' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Tele2 // Микрозаймы' if 
                                   x[:4]=='tele' or x[:2]=='t2' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Виджеты' if 
                                   x[:10]=='substitute' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Браузерные пуши (web)' if 
                                   x[:4]=='push' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Exbico' if 
                                   x[:6]=='exbico' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Ссылочная масса' if 
                                   x[:8]=='showcase' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Статьи на сайте' if 
                                   x[:12]=='content' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'МТС' if 
                                   x[:2]=='mt' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Brobank.ru' if 
                                   x[:7]=='brobank' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Тестирование гипотез' if 
                                  x[:6]=='yandex' or x[:6]=='google' else x)
# # df['utm_source'] = dr.apply(lambda row: 'Органический трафик' if row['utm_source']=='Ссылочная масса' 
# #                         else row['utm_source'], axis=1)
# df['utm_source'] = dr.apply(lambda row: 'New (2Gis)' if row['utm_source']=='2gis' 
#                         else row['utm_source'], axis=1)
# df['utm_source'] = dr.apply(lambda row: 'Мобильное приложение 1' if row['utm_source']=='Мобильное приложение' 
#                         else row['utm_source'], axis=1)

In [21]:
df

,week,utm_source,visits
0,2,Admitad,881.0
1,2,Adv.Cake,827.0
2,2,advertise,1.0
3,2,Перфоманс через Артикс,1044.0
4,2,Перфоманс через Артикс,2551.0
...,...,...,...
43,45,Прочее,4707.0
44,46,Прочее,4806.0
45,47,Прочее,5193.0
46,48,Прочее,6449.0


In [22]:
df['Отдел'] = df['utm_source']

In [23]:
df['Отдел'] = df['Отдел'].apply(lambda x: 'Direct' if 
                                   x in ['Email','SMS рекламные','Браузерные пуши (web)'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП',
                                         'накрутка поведенческих факторов',
                                         'Мобильное приложение 1','Мобильное приложение',
                                         'Новая карточка',
                                         'Новые витрины'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'CPA' if 
                                   x in ['Adv.Cake','Admitad','Leadgid','Cpahub','Rubid','Город Тройка',
                                         'Мобильное приложение',
                                         'Металлинвест \ потребы','CPA пиар','Новые продукты \\ дебетовые',
                                         'CallTat','Металлинвест \\ РКО','Промокоды','Чек Скан'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'Своя CPA' if 
                                   x in ['Bankiros.ru ссылки','Vbr.ru WL','Vbr.ru ссылки','Brobank.ru',
                                         'Металлинвест','Bankiros.ru WL','Новые партнеры WL'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'Контент' if 
                                   x in ['Яндекс.Дзен','Статьи на сайте',
                                         'Рекламные статьи (платные публикации)'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'Крупные партнеры' if 
                                   x in ['Tele2 // Микрозаймы','Теле2 \\ Кредитные карты',
                                         'Qiwi \\ Микрозайм','New (2Gis)','Мегафон','Крупный партнер (1)',
                                         'Крупный партнер (2)','Крупный партнер (3)'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'КЦ' if 
                                   x in ['Exbico','Внешние базы','Обзвон роботом','Допродажи КЦ'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'Перфоманс' if 
                                   x in ['Перфоманс','Перфоманс через Артикс','МТС \\  баннер + СМС',
                                         'Пуш партнерка','Тестирование гипотез'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'Прочее' if 
                                   x in ['stream_direct','propeller','evadav', 'Не определено'] else x)


In [24]:
df

,week,utm_source,visits,Отдел
0,2,Admitad,881.0,CPA
1,2,Adv.Cake,827.0,CPA
2,2,advertise,1.0,advertise
3,2,Перфоманс через Артикс,1044.0,Перфоманс
4,2,Перфоманс через Артикс,2551.0,Перфоманс
...,...,...,...,...
43,45,Прочее,4707.0,Прочее
44,46,Прочее,4806.0,Прочее
45,47,Прочее,5193.0,Прочее
46,48,Прочее,6449.0,Прочее


In [25]:
df = df[['week','visits','Отдел']]
df

,week,visits,Отдел
0,2,881.0,CPA
1,2,827.0,CPA
2,2,1.0,advertise
3,2,1044.0,Перфоманс
4,2,2551.0,Перфоманс
...,...,...,...
43,45,4707.0,Прочее
44,46,4806.0,Прочее
45,47,5193.0,Прочее
46,48,6449.0,Прочее


In [26]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:searchEngine",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000
    #filters = "ym:s:lastTrafficSource=='organic'"
# filters = "ym:s:UTMSource=='smsTopBP'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
#             'visit-id':result[i]["dimensions"][3]["name"],
            #'area':result[i]["dimensions"][1]["name"],
#             'users':result[i]["metrics"][0],
             'visits':result[i]["metrics"][0],
            #'gender':result[i]["dimensions"][2]["name"],
#'avgVisitDurationSeconds':result[i]["dimensions"][3]["name"],
           #'yandexClientId':result[i]["dimensions"][5]["name"]
            #'startURL':result[i]["dimensions"][7]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df_seo = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df_seo['date'] = pd.to_datetime(df_seo['date'])
df_seo['week'] =(df_seo['date'].dt.week)+1
df_seo = df_seo.groupby(['week','utm_source'],as_index=False).agg({'visits':'sum'})
df_seo = df_seo.groupby(['week'],as_index=False).agg({'visits':'sum'})

df_seo['Отдел']='SEO'
df_seo

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:57: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


,week,visits,Отдел
0,2,2130.0,SEO
1,3,2423.0,SEO
2,4,2369.0,SEO
3,5,1945.0,SEO
4,6,1770.0,SEO
5,7,1864.0,SEO
6,8,1965.0,SEO
7,9,1879.0,SEO
8,10,2148.0,SEO
9,11,1424.0,SEO


In [27]:
def get_kibana_screensplash(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 5000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"SPLASH_SCREEN"}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}},
"_source":["timestamp", "params.yandexClientID", "params.userId", "params.userId",
           "params.currentPage", "params.gaId","params.sourceGlobal"]}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.2)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/5000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-5000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr = dr[dr['params'].notnull()]
#     dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId','utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID', 'currentPage',
                 'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', 'payout', 'status',
                   'transaction_id', 'clickId', 'sourceGlobal'
                ]:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)
    del dr['params']

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['dt'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

In [28]:
since = date(2021,1,4)
till=date(2021,12,31)
screen_splash = get_kibana_screensplash(es, since,till)

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:8.255s]


34580; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:9.166s]


29580; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.890s]


...

14580; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:1.247s]


9580; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.418s]


4580; 

In [29]:
screen_splash['dt']=pd.to_datetime(screen_splash['dt'])
screen_splash['week'] = (screen_splash['dt'].dt.week)+1
dg1 = screen_splash.groupby(['week'], as_index=False).agg({'dt':'count'})
dg1['Отдел']='Мобильное приложение 1'
dg1.columns = ['week','visits','Отдел']
dg1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


,week,visits,Отдел
0,2,97,Мобильное приложение 1
1,3,87,Мобильное приложение 1
2,4,155,Мобильное приложение 1
3,5,173,Мобильное приложение 1
4,6,128,Мобильное приложение 1
5,7,96,Мобильное приложение 1
6,8,291,Мобильное приложение 1
7,9,282,Мобильное приложение 1
8,10,285,Мобильное приложение 1
9,11,455,Мобильное приложение 1


In [30]:
df_traf= pd.concat([df,df_seo])
df_traf = pd.concat([df_traf,dg1])
df_traf

,week,visits,Отдел
0,2,881.0,CPA
1,2,827.0,CPA
2,2,1.0,advertise
3,2,1044.0,Перфоманс
4,2,2551.0,Перфоманс
...,...,...,...
43,45,777.0,Мобильное приложение 1
44,46,825.0,Мобильное приложение 1
45,47,1008.0,Мобильное приложение 1
46,48,1489.0,Мобильное приложение 1


In [31]:
df_fin = df_traf.pivot_table(index='Отдел',columns='week',values='visits',aggfunc='sum')

df_fin = df_fin.fillna(0)

df_fin = df_fin.sort_index(axis='columns', level='week',ascending=False)


df_fin['sum'] = df_fin.sum(axis=1)


In [32]:
df_fin = df_fin[df_fin['sum']>500]
df_fin  = df_fin.sort_values('Отдел',ascending=False)
df_fin.to_excel('Недельный отчет/trafic_block_new_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(1) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/trafic_block_new_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)


In [33]:
df_fin

week,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,sum
Отдел,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Своя CPA,0.0,10.0,6.0,9.0,6.0,11.0,7.0,4.0,5.0,10.0,3.0,7.0,14.0,15.0,19.0,14.0,33.0,23.0,11.0,15.0,31.0,34.0,41.0,54.0,50.0,60.0,51.0,75.0,69.0,58.0,70.0,62.0,67.0,46.0,66.0,50.0,59.0,54.0,72.0,84.0,134.0,100.0,118.0,96.0,55.0,14.0,11.0,9.0,1912.0
Прочее,715.0,6452.0,5194.0,4811.0,4709.0,6262.0,6050.0,5539.0,5484.0,6453.0,5773.0,5352.0,4697.0,4192.0,4404.0,4125.0,3597.0,4190.0,3936.0,4561.0,1388.0,3313.0,3349.0,3337.0,3293.0,3497.0,5825.0,7541.0,4312.0,4436.0,4520.0,9132.0,13313.0,14006.0,16846.0,17531.0,16391.0,16396.0,12441.0,15892.0,14774.0,18076.0,18555.0,19051.0,18006.0,18781.0,18478.0,16475.0,415451.0
Перфоманс,72.0,574.0,775.0,724.0,724.0,1067.0,913.0,1655.0,1751.0,1245.0,1555.0,1375.0,1354.0,1344.0,1150.0,1180.0,1069.0,1038.0,1077.0,901.0,713.0,1038.0,998.0,2283.0,6275.0,3390.0,3109.0,3800.0,720.0,283.0,204.0,163.0,1245.0,2161.0,1115.0,316.0,246.0,200.0,172.0,192.0,239.0,362.0,328.0,297.0,147.0,129.0,326.0,3683.0,55677.0
Мобильное приложение 1,112.0,1489.0,1008.0,825.0,777.0,742.0,737.0,803.0,1085.0,1206.0,961.0,834.0,927.0,843.0,1071.0,1068.0,667.0,752.0,717.0,713.0,602.0,620.0,705.0,541.0,555.0,657.0,943.0,994.0,767.0,634.0,673.0,927.0,1576.0,1263.0,1574.0,904.0,692.0,489.0,455.0,285.0,282.0,291.0,96.0,128.0,173.0,155.0,87.0,97.0,34502.0
Крупные партнеры,197.0,12863.0,525.0,630.0,633.0,1049.0,4489.0,315.0,632.0,5072.0,86.0,101.0,193.0,468.0,4660.0,107.0,154.0,261.0,679.0,5457.0,175.0,254.0,700.0,7038.0,66.0,65.0,110.0,243.0,2603.0,114.0,156.0,443.0,3931.0,101.0,177.0,362.0,6237.0,45.0,53.0,62.0,82.0,629.0,724.0,439.0,15.0,24.0,15.0,23.0,63457.0
test,0.0,63.0,506.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,5.0,0.0,0.0,0.0,1.0,11.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,593.0
cityads,25.0,83.0,49.0,112.0,91.0,91.0,54.0,54.0,96.0,67.0,64.0,26.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,814.0
SEO,454.0,4768.0,4387.0,4844.0,4485.0,4925.0,4486.0,4646.0,4369.0,4095.0,3655.0,3320.0,2969.0,3025.0,3093.0,2787.0,2575.0,2545.0,2380.0,2032.0,531.0,1990.0,2159.0,2564.0,2447.0,2373.0,1984.0,1965.0,2237.0,2156.0,1994.0,2201.0,2524.0,2275.0,2065.0,2190.0,2493.0,2257.0,1424.0,2148.0,1879.0,1965.0,1864.0,1770.0,1945.0,2369.0,2423.0,2130.0,128162.0
Direct,11.0,488.0,1281.0,97.0,107.0,88.0,83.0,100.0,99.0,139.0,118.0,109.0,109.0,74.0,81.0,96.0,90.0,93.0,93.0,176.0,75.0,84.0,179.0,168.0,81.0,67.0,85.0,191.0,241.0,202.0,166.0,377.0,1038.0,330.0,451.0,606.0,1903.0,394.0,155.0,207.0,320.0,221.0,305.0,281.0,476.0,488.0,273.0,343.0,13239.0


In [34]:
def get_kibana_reg(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 1000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"REGISTRATION"}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/1000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-1000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID','currentPage',
#                 'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
#                   'transaction_id', 'clickId', 
                'sourceGlobal']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['dt_reg'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

regs = get_kibana_reg(es, since,till)


INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:0.033s]


33633; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.039s]


32633; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.034s]


...

2633; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.033s]


1633; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.024s]


633; 

In [35]:
regs['week'] = (pd.to_datetime(regs['dt_reg']).dt.week)+1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """Entry point for launching an IPython kernel.


In [36]:
regs['utm_source']=regs.apply(lambda row: row['utm_source'] if pd.notnull(row['utm_source']) else row['sourceGlobal'],axis=1)

In [37]:
regs['Отдел']= regs['utm_source']

In [38]:
regs['Отдел']=regs.apply(lambda row: row['Отдел'] if 
                      pd.notnull(row['Отдел']) else row['utm_medium'],axis=1)

In [39]:
regs['Отдел'] = regs.apply(lambda row: 'Мобильное приложение 1' if row['Отдел']=='Android_mobile' 
                        else row['Отдел'], axis=1)
regs['Отдел'] = regs.apply(lambda row: 'Мобильное приложение 1' if row['Отдел']=='Huawei_mobile' 
                        else row['Отдел'], axis=1)
regs['Отдел'] = regs.apply(lambda row: 'Мобильное приложение 1' if row['Отдел']=='Apple_mobile' 
                        else row['Отдел'], axis=1)
regs['Отдел'] = regs.apply(lambda row: 'SEO' if row['Отдел']=='organic' 
                        else row['Отдел'], axis=1)

In [40]:
regs['Отдел']=regs['Отдел'].fillna('Не определено')

In [41]:
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Vbr.ru ссылки' if 
                                  x in ['vbrru', 'upside_vbr','vbrr','vbr'] else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Email' if 
                                 x[:5]=='email' or x[:7]=='TRIGGER' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'SMS рекламные' if 
                                   x[:3]=='sms' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'SMS рекламные' if 
                                   x in ['tele2TommorowWON',
                                         'tele2Tommorow',
                                         'tele2FourDays',
                                         'tele2TOP'] or x[:5]=='t2sms' or x =='lostBasket' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Admitad' if 
                                   x[:7]=='admitad' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Adv.Cake' if 
                                   x[:7]=='advcake' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Leadgid' if 
                                   x[:7]=='leadgid' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Rubid' if 
                                   x[:5]=='rubid' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Cpahub' if 
                                   x[:6]=='cpahub' or x[:6]=='Cpahub' or x== 'get4click'
                                          or x =='checkscan' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Перфоманс через Артикс' if 
                                   x[:3]=='ais' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Tele2 // Микрозаймы' if 
                                   x[:4]=='tele' or x[:2]=='t2' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Виджеты' if 
                                   x[:10]=='substitute' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Браузерные пуши (web)' if 
                                   x[:4]=='push' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Exbico' if 
                                   x[:6]=='exbico' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Ссылочная масса' if 
                                   x[:8]=='showcase' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Статьи на сайте' if 
                                   x[:12]=='content' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'МТС' if 
                                   x[:2]=='mt' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Brobank.ru' if 
                                   x[:7]=='brobank' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Тестирование гипотез' if 
                                  x[:6]=='yandex' or x[:6]=='google' else x)
# # df['utm_source'] = dr.apply(lambda row: 'Органический трафик' if row['utm_source']=='Ссылочная масса' 
# #                      else row['utm_source'], axis=1)
# df['utm_source'] = dr.apply(lambda row: 'New (2Gis)' if row['utm_source']=='2gis' 
#                         else row['utm_source'], axis=1)
#df['utm_source'] = dr.apply(lambda row: 'Мобильное приложение 1' if row['sourceGlobal'] in ['Huawei_mobile','Apple_mobile','Android_mobile'] else row['Отдел'], axis=1)

In [42]:
df_reg = regs

In [43]:
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'Direct' if 
                                   x in ['Email','SMS рекламные','Браузерные пуши (web)'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП',
                                         'накрутка поведенческих факторов',
                                         'Новая карточка',
                                         'Новые витрины','content-text'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'CPA' if 
                                   x in ['Adv.Cake','Admitad','Leadgid','Cpahub','Rubid','Город Тройка',
                                         'Мобильное приложение',
                                         'Металлинвест \ потребы','CPA пиар','Новые продукты \\ дебетовые',
                                         'CallTat','Металлинвест \\ РКО','Промокоды','Чек Скан'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'Своя CPA' if 
                                   x in ['Bankiros.ru ссылки','Vbr.ru WL','Vbr.ru ссылки','Brobank.ru',
                                         'Металлинвест','Bankiros.ru WL','Новые партнеры WL'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'Контент' if 
                                   x in ['Яндекс.Дзен','Статьи на сайте',
                                         'Рекламные статьи (платные публикации)'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'Крупные партнеры' if 
                                   x in ['Tele2 // Микрозаймы','Теле2 \\ Кредитные карты',
                                         'Qiwi \\ Микрозайм','New (2Gis)','Мегафон','Крупный партнер (1)',
                                         'Крупный партнер (2)','Крупный партнер (3)','test_partner'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'КЦ' if 
                                   x in ['Exbico','Внешние базы','Обзвон роботом','Допродажи КЦ'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'Перфоманс' if 
                                   x in ['Перфоманс','Перфоманс через Артикс','МТС \\  баннер + СМС',
                                         'Пуш партнерка','Тестирование гипотез'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'Мобильное приложение 1 ' if 
                                   x in ['Мобильное приложение 1'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'Прочее' if 
                                   x in ['stream_direct','propeller','evadav', 'Не определено'] else x)


In [44]:
df_fin1 = df_reg.pivot_table(index='Отдел',columns='week',values='userId',aggfunc='count')

df_fin1 = df_fin1.fillna(0)

df_fin1 = df_fin1.sort_index(axis='columns', level='week',ascending=False)


df_fin1['sum'] = df_fin1.sum(axis=1)

In [45]:
df_fin1 = df_fin1[df_fin1['sum']>100]
df_fin1 = df_fin1.sort_values('Отдел',ascending=False)

In [46]:
df_fin1.to_excel('Недельный отчет/reg_new_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(2) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/reg_new_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)

In [47]:
df_fin1

week,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,sum
Отдел,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Своя CPA,1.0,30.0,30.0,36.0,22.0,17.0,30.0,23.0,19.0,33.0,19.0,17.0,25.0,31.0,34.0,42.0,45.0,41.0,40.0,42.0,54.0,59.0,43.0,61.0,53.0,84.0,131.0,147.0,174.0,145.0,98.0,166.0,189.0,137.0,218.0,208.0,194.0,162.0,126.0,108.0,109.0,147.0,137.0,139.0,141.0,142.0,146.0,70.0,4165.0
Прочее,5.0,57.0,69.0,74.0,65.0,64.0,44.0,55.0,53.0,69.0,58.0,54.0,47.0,58.0,55.0,66.0,58.0,69.0,83.0,160.0,138.0,143.0,173.0,167.0,126.0,177.0,266.0,293.0,244.0,263.0,196.0,222.0,248.0,222.0,243.0,288.0,284.0,219.0,177.0,199.0,165.0,178.0,160.0,162.0,136.0,165.0,138.0,123.0,6778.0
Перфоманс,0.0,3.0,1.0,1.0,4.0,0.0,0.0,1.0,6.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,5.0,1.0,2.0,5.0,3.0,5.0,1.0,6.0,17.0,6.0,13.0,18.0,11.0,8.0,3.0,9.0,9.0,11.0,13.0,6.0,12.0,8.0,6.0,3.0,7.0,9.0,7.0,6.0,2.0,2.0,4.0,57.0,296.0
Мобильное приложение 1,34.0,560.0,310.0,266.0,235.0,231.0,194.0,201.0,286.0,369.0,277.0,241.0,238.0,242.0,264.0,287.0,253.0,267.0,234.0,166.0,131.0,94.0,140.0,117.0,101.0,121.0,226.0,219.0,202.0,148.0,127.0,263.0,527.0,350.0,583.0,347.0,194.0,134.0,91.0,44.0,35.0,38.0,7.0,6.0,10.0,0.0,0.0,0.0,9410.0
Крупные партнеры,0.0,12.0,2.0,4.0,11.0,227.0,3.0,0.0,1.0,4.0,4.0,1.0,1.0,2.0,4.0,3.0,9.0,11.0,30.0,241.0,3.0,12.0,7.0,12.0,4.0,7.0,5.0,6.0,16.0,16.0,9.0,14.0,30.0,12.0,13.0,9.0,47.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,795.0
cityads,0.0,1.0,2.0,1.0,26.0,47.0,18.0,36.0,15.0,7.0,13.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,168.0
SEO,5.0,74.0,26.0,35.0,28.0,29.0,37.0,30.0,29.0,44.0,33.0,24.0,28.0,29.0,44.0,27.0,19.0,25.0,28.0,40.0,24.0,26.0,31.0,65.0,25.0,36.0,44.0,38.0,39.0,37.0,34.0,46.0,65.0,41.0,51.0,64.0,108.0,49.0,44.0,48.0,42.0,48.0,53.0,50.0,37.0,59.0,33.0,53.0,1924.0
Direct,0.0,2.0,3.0,3.0,4.0,2.0,3.0,6.0,4.0,3.0,1.0,2.0,6.0,1.0,3.0,1.0,3.0,2.0,1.0,2.0,3.0,0.0,1.0,0.0,0.0,2.0,3.0,5.0,7.0,6.0,6.0,5.0,10.0,6.0,8.0,8.0,29.0,11.0,9.0,3.0,9.0,2.0,8.0,0.0,10.0,9.0,10.0,7.0,229.0
CPA,21.0,210.0,215.0,325.0,406.0,290.0,323.0,258.0,316.0,744.0,580.0,426.0,444.0,94.0,67.0,36.0,39.0,93.0,90.0,122.0,146.0,132.0,138.0,142.0,92.0,123.0,124.0,136.0,138.0,194.0,346.0,178.0,106.0,102.0,185.0,235.0,217.0,162.0,128.0,148.0,172.0,153.0,157.0,131.0,168.0,170.0,141.0,188.0,9551.0


In [48]:
df_traf = df_traf.groupby(['Отдел','week'],as_index=False).agg({'visits':'sum'})
df_traf = pd.concat([df_traf,df_seo])
df_traf

,Отдел,week,visits
0,213,48,1.0
1,CPA,2,1831.0
2,CPA,3,2424.0
3,CPA,4,1736.0
4,CPA,5,1794.0
...,...,...,...
43,SEO,45,4485.0
44,SEO,46,4844.0
45,SEO,47,4387.0
46,SEO,48,4768.0


## Табличка трафик-рега

In [49]:
df_regs = df_reg.groupby(['Отдел','week'],as_index=False).agg({'userId':'count'})
df_regs = df_regs[df_regs['Отдел']!='']

In [50]:
cr_reg = df_regs.merge(df_traf,how='left',on=['Отдел','week'])

In [51]:
cr_reg['CR_Трафик-->Регистрация'] = ((cr_reg['userId']/cr_reg['visits'])*100).round(2)
cr_reg = cr_reg[cr_reg['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено','Контент','КЦ'])]

In [52]:
cr_reg_pivot = cr_reg.pivot_table(index='Отдел',
                                  columns='week'
                                  ,values='CR_Трафик-->Регистрация').sort_index(axis='columns',
                                                                                level='week',
                                                                                ascending=False)
cr_reg_pivot = cr_reg_pivot.fillna(0)
cr_reg_pivot = cr_reg_pivot.sort_values('Отдел',ascending=False)
cr_reg_pivot.to_excel('Недельный отчет/reg_traf_cr_new_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(3) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/reg_traf_cr_new_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)

In [53]:
cr_reg_pivot

week,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2
Отдел,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Своя CPA,0.00,300.00,500.00,400.00,366.67,154.55,428.57,575.00,380.00,330.00,633.33,242.86,178.57,206.67,178.95,300.00,136.36,178.26,363.64,280.00,174.19,173.53,104.88,112.96,106.00,140.00,256.86,196.00,252.17,250.00,140.00,267.74,282.09,297.83,330.30,416.00,328.81,300.00,175.00,128.57,81.34,147.00,116.10,144.79,256.36,1014.29,1327.27,777.78
Перфоманс,0.00,0.52,0.13,0.14,0.55,0.00,0.00,0.06,0.34,0.08,0.00,0.00,0.07,0.07,0.09,0.08,0.47,0.10,0.19,0.55,0.42,0.48,0.10,0.26,0.27,0.18,0.42,0.47,1.53,2.83,1.47,5.52,0.72,0.51,1.17,1.90,4.88,4.00,3.49,1.56,2.93,2.49,2.13,2.02,1.36,1.55,1.23,1.55
Крупные партнеры,0.00,0.09,0.38,0.63,1.74,21.64,0.07,0.00,0.16,0.08,4.65,0.99,0.52,0.43,0.09,2.80,5.84,4.21,4.42,4.42,1.71,4.72,1.00,0.17,6.06,10.77,4.55,2.47,0.61,14.04,5.77,3.16,0.76,11.88,7.34,2.49,0.75,0.00,0.00,0.00,0.00,0.16,0.00,0.00,0.00,0.00,13.33,0.00
Контент,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
КЦ,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.00,0.00,0.00,4.35,6.25,0.00,7.14,3.57,0.00,0.00,3.03,0.00
SEO,1.10,1.55,0.59,0.72,0.62,0.59,0.82,0.65,0.66,1.07,0.90,0.72,0.94,0.96,1.42,0.97,0.74,0.98,1.18,1.97,4.52,1.31,1.44,2.54,1.02,1.52,2.22,1.93,1.74,1.72,1.71,2.09,2.58,1.80,2.47,2.92,4.33,2.17,3.09,2.23,2.24,2.44,2.84,2.82,1.90,2.49,1.36,2.49
Direct,0.00,0.41,0.23,3.09,3.74,2.27,3.61,6.00,4.04,2.16,0.85,1.83,5.50,1.35,3.70,1.04,3.33,2.15,1.08,1.14,4.00,0.00,0.56,0.00,0.00,2.99,3.53,2.62,2.90,2.97,3.61,1.33,0.96,1.82,1.77,1.32,1.52,2.79,5.81,1.45,2.81,0.90,2.62,0.00,2.10,1.84,3.66,2.04
CPA,14.89,15.86,14.94,15.69,17.16,14.71,22.46,33.55,44.57,92.77,47.35,4.41,9.61,15.54,9.20,5.84,6.83,17.25,15.00,17.30,22.60,16.24,16.25,14.64,8.85,10.85,9.54,6.55,7.97,12.19,18.42,14.34,9.04,6.09,12.51,12.73,11.80,12.34,11.46,9.29,9.21,10.91,11.55,6.94,9.36,9.79,5.82,10.27


In [54]:
df = get_exact_claims(since, till, client)
df['requestId'] = df[df['partner']=='MoneyMan']['statusFields'].apply(mm_requestId)
dpbk = get_kibana_postback(es, since)
def float_apply(x):
    try:
        return float(x)
    except ValueError:
        return None
dpbk['payout'] = dpbk['payout'].apply(float_apply)
dpbk['payout'] = dpbk.apply(lambda x: x['payout']*1.2 if x['partnerName']=='admitad' else x['payout'], axis=1)
dpbk = dpbk[dpbk['status'].isin(['APPROVED', 'approved', 'ISSUED'])].sort_values('payout').drop_duplicates('transitionId', keep='last')

dr = pd.merge(left=df, right=dpbk, left_on='id', right_on='transitionId', how='left')
dc = get_kibana_claims(es, since,till)[['clickId', 'userId', 'utm_campaign', 'utm_medium', 'utm_content', 
                                        'utm_source', 'utm_term', 'gaId', 'yandexClientID', 'sourceGlobal']]
dr = pd.merge(left=dr, right=dc[dc['clickId'].notnull()].drop_duplicates('clickId'), on='clickId', how='left')

cre = cre_get(es, since)
dr['score'] = dr['userId_x'].map(cre).fillna(0).astype(int)

dr.to_pickle(f'reports/{datetime.strftime(since, "%B")}_claimReport.pickle.gz')
dc.to_pickle(f'reports/{datetime.strftime(since, "%B")}_claims.pickle.gz')

def AR_count(score, productType, status):
    koef = 1
    check = [2200, 1700, 1100, 5500]
    ARlist = [[0.00499500499500499, 0.00305343511450382, 0.0714285714285714, 0.0], 
              [0.00380818053596615, 0.00871508379888268, 0.0768345323741007, 0.0], 
              [0.00685378590078329, 0.0136418349605457, 0.0846340617412418, 0.0053475935828877], 
              [0.0111028867505552, 0.0255610972568579, 0.0980751604032997, 0.0116279069767442], 
              [0.00938967136150235, 0.0349985043374215, 0.126042632066728, 0.0], 
              [0.0109689213893967, 0.0407338308457712, 0.140204271123491, 0.0], 
              [0.0141884222474461, 0.0514050719671008, 0.113486842105263, 0.0188679245283019], 
              [0.0150753768844221, 0.0687361419068736, 0.0743243243243243, 0.0], 
              [0.00943396226415094, 0.0415430267062315, 0.0892857142857143, 0.0],
              [0.00943396226415094, 0.0415430267062315, 0.0892857142857143, 0.0]]
    
    if score is np.nan or status in ['NEW', 'ERROR']:
        return 0
    elif score > 10 and productType == 'CASH_LOAN':
        return round(ARlist[int(round(score, -1)/10)-1][0]*check[0]*koef, 2)
    elif score > 10 and productType == 'CREDIT_CARD':
        return round(ARlist[int(round(score, -1)/10)-1][1]*check[1]*koef, 2)
    elif score > 10 and productType == 'LOAN':
#         print(score, productType, status)
        return round(ARlist[int(round(score, -1)/10)-1][2]*check[2]*koef, 2)
    elif score > 10 and productType == 'REFINANCE':
        return round(ARlist[int(round(score, -1)/10)-1][3]*check[3]*koef, 2)
    else:
        return round(ARlist[int(round(score, -1)/10)-1][0]*check[0]*koef, 2)

dr['createdAt'] = pd.to_datetime(dr['createdAt'])
dr['payout'] = dr['payout'].astype(float)
# dr['payout'] = dr.apply(lambda row: 3800 if row['partner']=='Альфа-Банк' 
#                         else row['payout'], axis=1)
# dr['payout'] = dr.apply(lambda row: 2550 if row['partner']=='Kredito24' 
#                         else row['payout'], axis=1)
dr['productType'] = dr['product'].apply(lambda x: x['productType'])


dr['source'] = dr.apply(lambda row: 'Металинвест Cash' if row['productId_x']=='5e9846be79dd27000141f476' 
                        else row['source'], axis=1)

#dr = dr[dr['status_x']=='NEW']
dr['score'] = dr['score'].apply(lambda x: x+75 if x==0 else x)
dr['whence']=dr.apply(lambda row: row['whence_y'] if pd.notnull(row['whence_y']) else row['whence_x'],axis=1)
dr['source']=dr.apply(lambda row: row['utm_source_y'] if pd.notnull(row['utm_source_y']) else row['whence'],axis=1)
dr['medium']=dr.apply(lambda row: row['utm_medium_y'] if pd.notnull(row['utm_medium_y']) else row['whence'],axis=1)

sms = dr.groupby('source').agg({'_id':'count'}).to_excel('sms.xlsx')

dr['source']=dr.apply(lambda row: row['utm_source_y'] if 
                      pd.notnull(row['utm_source_y']) else row['whence'],axis=1)
dr['source']=dr.apply(lambda row: row['sourceGlobal'] if 
                      pd.notnull(row['sourceGlobal']) else row['source'],axis=1)
dr['source']=dr.apply(lambda row: 'Город Тройка' if 
                      row['utm_content']=='gorodtroika' else row['source'],axis=1)
dr['source'] = dr['source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
dr['source'] = dr['source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
dr['source'] = dr['source'].apply(lambda x: 'Vbr.ru ссылки' if 
                                  x in ['vbrru', 'upside_vbr','vbrr'] else x)
dr['source'] = dr['source'].apply(lambda x: 'Email' if 
                                 x[:5]=='email' or x[:7]=='TRIGGER' else x)
dr['source'] = dr['source'].apply(lambda x: 'SMS рекламные' if 
                                   x[:3]=='sms' else x)
dr['source'] = dr['source'].apply(lambda x: 'SMS рекламные' if 
                                   x in ['tele2TommorowWON',
                                         'tele2Tommorow',
                                         'tele2FourDays',
                                         'tele2TOP'] or x[:5]=='t2sms' or x =='lostBasket' else x)
dr['source'] = dr['source'].apply(lambda x: 'Admitad' if 
                                   x[:7]=='admitad' else x)
dr['source'] = dr['source'].apply(lambda x: 'Adv.Cake' if 
                                   x[:7]=='advcake' else x)
dr['source'] = dr['source'].apply(lambda x: 'Leadgid' if 
                                   x[:7]=='leadgid' else x)
dr['source'] = dr['source'].apply(lambda x: 'Rubid' if 
                                   x[:5]=='rubid' else x)
dr['source'] = dr['source'].apply(lambda x: 'Cpahub' if 
                                   x[:6]=='cpahub' or x[:6]=='Cpahub' or x== 'get4click'  else x)
dr['source'] = dr['source'].apply(lambda x: 'Перфоманс через Артикс' if 
                                   x[:3]=='ais' else x)
dr['source'] = dr['source'].apply(lambda x: 'Tele2 // Микрозаймы' if 
                                   x[:4]=='tele' else x)
dr['source'] = dr['source'].apply(lambda x: 'Виджеты' if 
                                   x[:10]=='substitute' else x)
dr['source'] = dr['source'].apply(lambda x: 'Браузерные пуши (web)' if 
                                   x[:4]=='push' else x)
dr['source'] = dr['source'].apply(lambda x: 'Exbico' if 
                                   x[:6]=='exbico' else x)
dr['source'] = dr['source'].apply(lambda x: 'Ссылочная масса' if 
                                   x[:8]=='showcase' else x)
dr['source'] = dr['source'].apply(lambda x: 'Статьи на сайте' if 
                                   x[:12]=='content-text' else x)
dr['source'] = dr['source'].apply(lambda x: 'МТС' if 
                                   x[:2]=='mt' else x)
dr['source'] = dr['source'].apply(lambda x: 'Brobank.ru' if 
                                   x[:7]=='brobank' else x)
dr['source'] = dr['source'].apply(lambda x: 'Тестирование гипотез' if 
                                  x[:6]=='yandex' or x[:6]=='google' else x)
dr['source']=dr.apply(lambda row: 'Тестирование гипотез' if 
                      row['utm_source_y']=='yandex' or row['utm_source_y']=='google' else row['source'],axis=1)

dr['source'] = dr.apply(lambda row: 'Металлинвест \\ потребы' if row['productId_x']=='5e9846be79dd27000141f476' 
                        else row['source'], axis=1)
dr['source'] = dr.apply(lambda row: 'Металлинвест \\ потребы' if row['productId_x']=='5ef4473acf22a90001010445' 
                        else row['source'], axis=1)


def get_kibana_reg(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 1000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"REGISTRATION"}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/1000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-1000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID','currentPage',
#                 'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
#                   'transaction_id', 'clickId', 
                'sourceGlobal']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['dt_reg'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

regs = get_kibana_reg(es, since,till)

dr['utm_source_x'].value_counts()

regs['currentPage1'] = regs['currentPage'].apply(lambda x: x[:5] if pd.notnull(x) else np.nan)
regs['currentPage2'] = regs['currentPage'].apply(lambda x: x[:9] if pd.notnull(x) else np.nan)

dr=pd.merge(left=dr, right=regs, how='left', on='userId')

odin = dr[dr['dt_reg'].notnull()]
odin=odin[odin['product_type']=='LOAN']
odin['userId'].nunique()

dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Android_mobile' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Huawei_mobile' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Apple_mobile' 
                        else row['source_x'], axis=1)

dr['source_x'] = dr.apply(lambda row: 'Статьи на сайте' if row['currentPage1']=='/news' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Статьи на сайте' if row['currentPage2']=='/articles' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Органический трафик' if row['source_x']=='Ссылочная масса' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'New (2Gis)' if row['source_x']=='2gis' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение 1' if row['source_x']=='Мобильное приложение' 
                        else row['source_x'], axis=1)
             
dr['status_x'] = dr.apply(lambda x: 'APPROVED' if
                        pd.notnull(x['status_y']) and x['status_y'] in(['APPROVED', 'approved', 'ISSUED']) 
                          else x['status_x'], axis=1)


273503


INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:1.214s]


554784; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.269s]


544784; 

...

2634; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.035s]


1634; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.025s]


634; 

In [55]:
dr.head(1)

,_id,type,createdAt,updatedAt,status_x,borrower,partner,product,statusFields,whence_x,adapterName,clickId,id_x,dt,productId_x,product_type,product_name,userId_x,phone,requestId,indexName_x,fallbackResponse_x,json_x,id_y,eventType_x,source_x,params_x,timestamp_x,gaId_x,userId_y,utm_campaign_x,utm_medium_x,utm_source_x,utm_term_x,productType_x,yandexClientID_x,productId_y,whence_y,transitionId,partnerName,payout,status_y,transaction_id,externalId,orderId,transactionId,time_x,date,userId,utm_campaign_y,utm_medium_y,utm_content,utm_source_y,utm_term_y,gaId_y,yandexClientID_y,sourceGlobal_x,score,whence,medium,indexName_y,fallbackResponse_y,json_y,id,eventType_y,source_y,params_y,timestamp_y,gaId,utm_campaign,utm_medium,utm_source,utm_term,productType_y,yandexClientID,currentPage,sourceGlobal_y,time_y,dt_reg,currentPage1,currentPage2
0,5ff2312053668d0001f5b3b9,LINK,2021-01-03 21:03:28.953,2021-01-03 21:03:28.953,NEW,"{'_id': None, 'userId': '5ff231209bad3e0001d75cc9'}",Тинькофф Банк,"{'_id': 5dcd77050d00b700010cfb9c, 'name': 'Tinkoff Drive', 'productType': 'CREDIT_CARD', 'productName': 'Кредитная карта'}",{},vbrru,NaN,NaN,5ff2312053668d0001f5b3b9,2021-01-04,5dcd77050d00b700010cfb9c,CREDIT_CARD,Tinkoff Drive,5ff231209bad3e0001d75cc9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vbr.ru ссылки,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT_CARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,vbrru,vbrru,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN


In [56]:
dr['dt'] = pd.to_datetime(dr['dt'])
dr['week'] = (dr['dt'].dt.week)+1
dr['Отдел'] = dr['source_x']
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'Direct' if 
                                   x in ['Email','SMS рекламные','Браузерные пуши (web)'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП',
                                         'накрутка поведенческих факторов',
                                         'Мобильное приложение',
                                         'Новая карточка',
                                         'Новые витрины'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'CPA' if 
                                   x in ['Adv.Cake','Admitad','Leadgid','Cpahub','Rubid','Город Тройка',
                                         'Мобильное приложение',
                                         'Металлинвест \ потребы','CPA пиар','Новые продукты \\ дебетовые',
                                         'CallTat','Металлинвест \\ РКО','Промокоды','Чек Скан'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'Своя CPA' if 
                                   x in ['Bankiros.ru ссылки','Vbr.ru WL','Vbr.ru ссылки','Brobank.ru',
                                         'Металлинвест','Bankiros.ru WL','Новые партнеры WL'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'Контент' if 
                                   x in ['Яндекс.Дзен','Статьи на сайте',
                                         'Рекламные статьи (платные публикации)'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'Крупные партнеры' if 
                                   x in ['Tele2 // Микрозаймы','Теле2 \\ Кредитные карты',
                                         'Qiwi \\ Микрозайм','New (2Gis)','Мегафон','Крупный партнер (1)',
                                         'Крупный партнер (2)','Крупный партнер (3)'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'КЦ' if 
                                   x in ['Exbico','Внешние базы','Обзвон роботом','Допродажи КЦ'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'Перфоманс' if 
                                   x in ['Перфоманс','Тендерное агентство (Артикс)','МТС \\  баннер + СМС',
                                         'Пуш партнерка','Тестирование гипотез'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'Мобильное приложение 1' if 
                                   x in ['Мобильное приложение 1'] else x)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [57]:
dr_kliki = dr[dr['status_x']=='NEW'].groupby(['Отдел','week','product_type'],as_index=False).agg({'dt':'count'})
dr_kliki['Отдел'].unique()
dr_kliki = dr_kliki[dr_kliki['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено','Контент','КЦ'])]
dr_kliki_pivot = dr_kliki.pivot_table(index=['Отдел','product_type'],
                                      columns='week',
                                      values='dt').sort_index(axis='columns',
                                                                                level='week',
                                                                                ascending=False)
dr_kliki_pivot=dr_kliki_pivot.fillna(0)
dr_kliki_pivot = dr_kliki_pivot.reset_index().copy()
dr_kliki_pivot =dr_kliki_pivot.sort_values('Отдел',ascending=False)
dr_kliki_pivot.to_excel('Недельный отчет/Kliki_new_Alexander_raz.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(4) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/Kliki_new_Alexander_raz.xlsx'
                  )
set_with_dataframe(worksheet, df)
dr_kliki_pivot

week,Отдел,product_type,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2
75,Своя CPA,REFINANCE,0.0,2.0,4.0,8.0,3.0,2.0,0.0,1.0,4.0,3.0,0.0,0.0,0.0,4.0,6.0,4.0,5.0,1.0,5.0,10.0,2.0,2.0,3.0,1.0,10.0,7.0,11.0,13.0,23.0,15.0,11.0,5.0,17.0,17.0,14.0,21.0,21.0,15.0,4.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74,Своя CPA,LOAN,47.0,493.0,544.0,545.0,753.0,710.0,691.0,602.0,612.0,590.0,572.0,572.0,623.0,650.0,592.0,659.0,622.0,491.0,536.0,625.0,712.0,731.0,760.0,900.0,925.0,1085.0,1602.0,2208.0,2120.0,1532.0,1297.0,1110.0,1582.0,2114.0,2207.0,2330.0,3769.0,3806.0,3612.0,3406.0,4078.0,3473.0,2857.0,2511.0,4406.0,4430.0,4293.0,3021.0
73,Своя CPA,CREDIT_CARD,0.0,7.0,11.0,2.0,6.0,3.0,8.0,7.0,4.0,6.0,8.0,5.0,1.0,10.0,14.0,6.0,17.0,12.0,18.0,8.0,30.0,22.0,14.0,19.0,34.0,81.0,80.0,96.0,55.0,79.0,83.0,86.0,106.0,103.0,73.0,89.0,67.0,68.0,57.0,52.0,52.0,88.0,57.0,121.0,113.0,114.0,105.0,114.0
72,Своя CPA,CASH_LOAN,3.0,14.0,18.0,21.0,3.0,3.0,17.0,2.0,4.0,14.0,9.0,7.0,9.0,8.0,11.0,13.0,12.0,7.0,7.0,9.0,7.0,13.0,3.0,12.0,5.0,12.0,18.0,26.0,39.0,27.0,12.0,29.0,34.0,30.0,53.0,37.0,83.0,51.0,24.0,12.0,5.0,7.0,25.0,14.0,14.0,10.0,30.0,7.0
71,Своя CPA,AUTOCREDIT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65,Перфоманс,CREDIT_PLEDGE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62,Перфоманс,AUTOCREDIT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
63,Перфоманс,CASH_LOAN,0.0,2.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,1.0,1.0,0.0,1.0,2.0,0.0,1.0,2.0,6.0,5.0,0.0,0.0,4.0,1.0,3.0,2.0,3.0,7.0,7.0,2.0,4.0,5.0,5.0,4.0,2.0,1.0,1.0,2.0,7.0,3.0,4.0,2.0,11.0,4.0,7.0,5.0,0.0,1.0,2.0
64,Перфоманс,CREDIT_CARD,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,6.0,2.0,4.0,7.0,1.0,0.0,0.0,0.0,3.0,3.0,1.0,0.0,5.0,3.0,3.0,5.0,1.0,5.0,1.0,4.0,4.0,0.0,2.0,1.0
66,Перфоманс,DEBIT_CARD,0.0,0.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,2.0,1.0,3.0,11.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [58]:
dr_kliki = dr[dr['status_x']=='NEW'].groupby(['Отдел','week'],as_index=False).agg({'dt':'count'})
dr_kliki['Отдел'].unique()
dr_kliki = dr_kliki[dr_kliki['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено','Контент','КЦ'])]
dr_kliki_pivot = dr_kliki.pivot_table(index=['Отдел'],
                                      columns='week',
                                      values='dt').sort_index(axis='columns',
                                                                                level='week',
                                                                                ascending=False)
dr_kliki_pivot=dr_kliki_pivot.fillna(0)
dr_kliki_pivot =dr_kliki_pivot.sort_values('Отдел',ascending=False)
dr_kliki_pivot = dr_kliki_pivot.reset_index().copy()
dr_kliki_pivot.to_excel('Недельный отчет/Kliki_new_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(5) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/Kliki_new_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
dr_kliki_pivot

week,Отдел,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2
0,Своя CPA,50.0,516.0,577.0,576.0,765.0,718.0,716.0,612.0,624.0,613.0,589.0,584.0,633.0,672.0,623.0,682.0,656.0,511.0,566.0,652.0,751.0,768.0,780.0,932.0,974.0,1185.0,1711.0,2344.0,2237.0,1653.0,1403.0,1231.0,1739.0,2264.0,2347.0,2477.0,3940.0,3940.0,3697.0,3480.0,4135.0,3568.0,2939.0,2646.0,4533.0,4554.0,4428.0,3142.0
1,Перфоманс,0.0,6.0,18.0,24.0,17.0,47.0,42.0,30.0,24.0,12.0,17.0,8.0,19.0,20.0,27.0,23.0,15.0,17.0,18.0,1.0,2.0,22.0,4.0,16.0,56.0,44.0,61.0,120.0,15.0,6.0,5.0,16.0,12.0,6.0,9.0,47.0,16.0,20.0,7.0,34.0,6.0,21.0,5.0,11.0,32.0,5.0,5.0,6.0
2,Мобильное приложение 1,34.0,719.0,473.0,377.0,364.0,346.0,332.0,372.0,438.0,517.0,429.0,333.0,424.0,395.0,520.0,580.0,437.0,391.0,337.0,241.0,212.0,202.0,287.0,244.0,190.0,221.0,387.0,418.0,369.0,277.0,301.0,482.0,767.0,524.0,742.0,509.0,276.0,220.0,138.0,89.0,89.0,63.0,11.0,23.0,46.0,0.0,0.0,0.0
3,Крупные партнеры,59.0,1753.0,50.0,70.0,65.0,135.0,910.0,33.0,131.0,1439.0,29.0,11.0,44.0,122.0,1409.0,37.0,34.0,43.0,150.0,1457.0,30.0,66.0,309.0,2456.0,39.0,32.0,14.0,126.0,964.0,107.0,83.0,242.0,1912.0,104.0,152.0,402.0,4519.0,27.0,6.0,35.0,47.0,299.0,366.0,186.0,12.0,33.0,9.0,16.0
4,Контент,0.0,8.0,1.0,8.0,5.0,7.0,2.0,7.0,7.0,6.0,5.0,0.0,3.0,3.0,0.0,1.0,7.0,4.0,4.0,5.0,11.0,14.0,11.0,4.0,9.0,14.0,12.0,11.0,8.0,19.0,24.0,14.0,17.0,7.0,123.0,42.0,143.0,14.0,16.0,7.0,6.0,13.0,6.0,3.0,14.0,18.0,18.0,19.0
5,КЦ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,3.0,0.0,1.0,3.0,1.0,2.0,0.0,0.0
6,SEO,61.0,413.0,292.0,221.0,238.0,205.0,301.0,209.0,236.0,299.0,254.0,232.0,284.0,210.0,282.0,289.0,179.0,309.0,324.0,504.0,402.0,399.0,473.0,461.0,535.0,370.0,610.0,946.0,481.0,420.0,347.0,630.0,753.0,581.0,751.0,734.0,693.0,657.0,451.0,425.0,326.0,466.0,291.0,444.0,321.0,369.0,352.0,430.0
7,Direct,69.0,1654.0,645.0,387.0,522.0,581.0,401.0,434.0,513.0,886.0,530.0,599.0,546.0,1165.0,1147.0,1083.0,568.0,507.0,1156.0,2077.0,606.0,535.0,1611.0,3493.0,590.0,642.0,1025.0,873.0,668.0,741.0,544.0,972.0,1458.0,1044.0,1001.0,826.0,1740.0,1150.0,515.0,1009.0,283.0,565.0,1074.0,963.0,1131.0,950.0,1009.0,285.0
8,CPA,40.0,529.0,574.0,667.0,880.0,574.0,705.0,669.0,520.0,580.0,527.0,1008.0,577.0,542.0,523.0,727.0,532.0,463.0,440.0,711.0,542.0,550.0,624.0,643.0,583.0,618.0,766.0,839.0,777.0,943.0,869.0,731.0,532.0,659.0,769.0,1103.0,1225.0,1263.0,1177.0,1420.0,1069.0,1252.0,1438.0,1426.0,1136.0,646.0,552.0,425.0


In [59]:
dr_zayavki = dr[dr['status_x']!='NEW'].groupby(['Отдел','week','product_type'],as_index=False).agg({'dt':'count'})
dr_zayavki['Отдел'].unique()
dr_zayavki = dr_zayavki[dr_zayavki['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено','Контент','КЦ'])]
dr_zayavki_pivot = dr_zayavki.pivot_table(index=['Отдел','product_type'],
                                      columns='week',
                                      values='dt',aggfunc='sum').sort_index(axis='columns',
                                                                                level='week',
                                                                                ascending=False)
dr_zayavki_pivot=dr_zayavki_pivot.fillna(0)
dr_zayavki_pivot =dr_zayavki_pivot.sort_values('Отдел',ascending=False)

dr_zayavki_pivot = dr_zayavki_pivot.reset_index().copy()
dr_zayavki_pivot.to_excel('Недельный отчет/Zayavki_Alexander_raz.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(6) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/Zayavki_Alexander_raz.xlsx'
                  )
set_with_dataframe(worksheet, df)
dr_zayavki_pivot

week,Отдел,product_type,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2
0,Своя CPA,REFINANCE,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0,3.0,0.0,1.0,0.0,2.0,4.0,1.0,2.0,1.0,0.0,0.0,7.0,7.0,19.0,25.0,10.0,16.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Своя CPA,LOAN,7.0,74.0,43.0,37.0,70.0,82.0,61.0,59.0,56.0,142.0,161.0,113.0,141.0,168.0,179.0,164.0,171.0,170.0,173.0,187.0,193.0,216.0,128.0,68.0,74.0,94.0,144.0,197.0,255.0,96.0,90.0,60.0,125.0,226.0,197.0,214.0,273.0,562.0,469.0,474.0,270.0,386.0,391.0,326.0,225.0,213.0,213.0,340.0
2,Своя CPA,CREDIT_CARD,0.0,1.0,7.0,4.0,4.0,3.0,4.0,0.0,1.0,3.0,0.0,4.0,2.0,5.0,7.0,4.0,5.0,8.0,2.0,3.0,6.0,4.0,9.0,7.0,3.0,10.0,16.0,21.0,38.0,25.0,28.0,26.0,28.0,32.0,60.0,73.0,58.0,56.0,31.0,35.0,44.0,42.0,32.0,29.0,46.0,34.0,30.0,19.0
3,Своя CPA,CASH_LOAN,1.0,4.0,3.0,12.0,8.0,2.0,8.0,0.0,2.0,2.0,6.0,1.0,3.0,5.0,3.0,1.0,6.0,5.0,2.0,4.0,6.0,5.0,6.0,7.0,8.0,5.0,16.0,18.0,22.0,11.0,3.0,13.0,19.0,15.0,22.0,16.0,21.0,11.0,25.0,7.0,10.0,11.0,8.0,12.0,14.0,7.0,19.0,3.0
4,Перфоманс,RKO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Перфоманс,REFINANCE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Перфоманс,MORTGAGE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Перфоманс,LOAN,0.0,0.0,0.0,7.0,8.0,14.0,7.0,7.0,7.0,3.0,0.0,1.0,0.0,3.0,3.0,5.0,2.0,0.0,0.0,1.0,0.0,5.0,1.0,1.0,8.0,12.0,11.0,18.0,0.0,2.0,0.0,2.0,5.0,2.0,1.0,5.0,3.0,0.0,1.0,1.0,1.0,0.0,2.0,0.0,2.0,0.0,0.0,1.0
8,Перфоманс,DEBIT_CARD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Перфоманс,CREDIT_CARD,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,2.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,3.0,0.0,1.0


In [60]:
dr_zayavki = dr[dr['status_x']!='NEW'].groupby(['Отдел','week','product_type'],as_index=False).agg({'dt':'count'})
dr_zayavki['Отдел'].unique()
dr_zayavki = dr_zayavki[dr_zayavki['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено','Контент','КЦ'])]
dr_zayavki_pivot = dr_zayavki.pivot_table(index=['Отдел'],
                                      columns='week',
                                      values='dt',aggfunc='sum').sort_index(axis='columns',
                                                                                level='week',
                                                                                ascending=False)
dr_zayavki_pivot=dr_zayavki_pivot.fillna(0)
dr_zayavki_pivot =dr_zayavki_pivot.sort_values('Отдел',ascending=False)

dr_zayavki_pivot = dr_zayavki_pivot.reset_index().copy()
dr_zayavki_pivot.to_excel('Недельный отчет/Zayavki_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(7) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/Zayavki_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
dr_zayavki_pivot

week,Отдел,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2
0,Своя CPA,8.0,79.0,53.0,54.0,82.0,87.0,73.0,59.0,61.0,148.0,168.0,118.0,146.0,178.0,189.0,170.0,182.0,184.0,179.0,195.0,208.0,225.0,144.0,82.0,87.0,113.0,177.0,238.0,316.0,132.0,121.0,106.0,179.0,292.0,304.0,313.0,368.0,635.0,525.0,516.0,324.0,439.0,431.0,367.0,285.0,254.0,262.0,362.0
1,Перфоманс,0.0,2.0,0.0,7.0,8.0,14.0,7.0,8.0,8.0,5.0,1.0,1.0,0.0,4.0,3.0,6.0,9.0,3.0,4.0,1.0,2.0,6.0,4.0,4.0,13.0,13.0,18.0,21.0,2.0,2.0,4.0,10.0,9.0,14.0,8.0,11.0,11.0,1.0,1.0,3.0,3.0,8.0,10.0,6.0,6.0,3.0,2.0,2.0
2,Мобильное приложение 1,13.0,169.0,116.0,107.0,129.0,117.0,110.0,96.0,141.0,137.0,128.0,107.0,95.0,90.0,80.0,97.0,99.0,91.0,56.0,35.0,38.0,54.0,63.0,40.0,29.0,25.0,60.0,59.0,47.0,40.0,45.0,62.0,147.0,146.0,211.0,112.0,79.0,50.0,34.0,16.0,18.0,14.0,2.0,1.0,8.0,0.0,0.0,0.0
3,Крупные партнеры,8.0,355.0,9.0,3.0,15.0,42.0,192.0,7.0,35.0,265.0,10.0,0.0,6.0,18.0,336.0,5.0,10.0,17.0,58.0,439.0,10.0,11.0,58.0,432.0,5.0,1.0,1.0,14.0,134.0,5.0,5.0,29.0,186.0,6.0,20.0,30.0,306.0,0.0,2.0,3.0,11.0,21.0,21.0,19.0,0.0,1.0,3.0,0.0
4,Контент,0.0,4.0,1.0,9.0,1.0,3.0,0.0,1.0,2.0,2.0,4.0,3.0,3.0,1.0,0.0,0.0,3.0,2.0,1.0,4.0,2.0,7.0,5.0,3.0,3.0,5.0,4.0,2.0,3.0,1.0,4.0,2.0,4.0,7.0,9.0,18.0,45.0,3.0,5.0,1.0,11.0,5.0,2.0,4.0,2.0,14.0,5.0,4.0
5,КЦ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,SEO,10.0,89.0,52.0,56.0,61.0,54.0,105.0,52.0,47.0,58.0,57.0,44.0,52.0,50.0,62.0,53.0,39.0,88.0,53.0,68.0,54.0,96.0,100.0,137.0,121.0,106.0,126.0,128.0,131.0,132.0,88.0,110.0,152.0,182.0,177.0,249.0,194.0,143.0,122.0,114.0,114.0,153.0,108.0,122.0,71.0,95.0,81.0,65.0
7,Direct,3.0,109.0,104.0,37.0,48.0,50.0,53.0,69.0,63.0,85.0,51.0,40.0,42.0,93.0,110.0,83.0,75.0,87.0,68.0,155.0,80.0,74.0,128.0,261.0,81.0,72.0,177.0,71.0,71.0,62.0,80.0,90.0,134.0,135.0,118.0,126.0,225.0,155.0,84.0,126.0,50.0,69.0,172.0,162.0,237.0,94.0,249.0,53.0
8,CPA,14.0,92.0,96.0,120.0,168.0,89.0,156.0,104.0,55.0,75.0,69.0,124.0,98.0,76.0,75.0,44.0,51.0,61.0,71.0,104.0,73.0,67.0,103.0,143.0,119.0,122.0,159.0,198.0,180.0,238.0,305.0,226.0,99.0,116.0,137.0,199.0,275.0,247.0,225.0,273.0,251.0,272.0,391.0,346.0,319.0,176.0,155.0,112.0


In [61]:
cr_klik_zayavka = dr_zayavki.merge(dr_kliki,how='left',on=['Отдел','week'])
cr_klik_zayavka
cr_klik_zayavka['cr_klik_zayavka_'] = ((cr_klik_zayavka['dt_x']/cr_klik_zayavka['dt_y'])*100).round(2)
cr_klik_zayavka_pivot =cr_klik_zayavka.pivot_table(index=['Отдел'],
                                                   columns='week',
                                                   values='cr_klik_zayavka_').sort_index(axis='columns',
                                                                                level='week',
                                                                                ascending=False)
cr_klik_zayavka_pivot = cr_klik_zayavka_pivot.fillna(0)
cr_klik_zayavka_pivot =cr_klik_zayavka_pivot.sort_values('Отдел',ascending=False)
cr_klik_zayavka_pivot = cr_klik_zayavka_pivot.reset_index().copy()

cr_klik_zayavka_pivot.to_excel('Недельный отчет/cr_kliki_Zayavki_Alexander_raz.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(8) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/cr_kliki_Zayavki_Alexander_raz.xlsx'
                  )
set_with_dataframe(worksheet, df)
cr_klik_zayavka_pivot

week,Отдел,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2
0,Своя CPA,8.000000,5.103333,3.060000,2.340000,3.573333,4.040000,3.400000,9.640000,2.442500,6.035000,9.506667,6.733333,7.686667,8.826667,10.1100,6.235000,9.246667,9.005000,7.9050,7.475000,6.925000,9.763333,4.615000,2.933333,2.235000,2.382500,2.590000,2.54000,3.530000,2.663333,2.873333,2.152500,2.572500,3.222500,3.240000,3.160000,2.335000,4.027500,4.736667,4.943333,2.610000,4.103333,4.886667,4.623333,2.093333,1.8600,1.973333,3.840000
1,Перфоманс,0.000000,16.670000,0.000000,29.170000,47.060000,29.790000,16.670000,13.330000,16.670000,20.835000,5.880000,12.500000,0.000000,10.000000,11.1100,13.045000,19.996667,17.650000,11.1150,100.000000,50.000000,13.640000,33.333333,8.333333,7.740000,14.770000,9.836667,8.75000,13.330000,33.330000,80.000000,15.625000,25.003333,77.776667,22.220000,5.852500,22.916667,5.000000,14.290000,4.410000,16.670000,38.100000,66.666667,27.270000,6.250000,60.0000,40.000000,16.670000
2,Мобильное приложение 1,19.120000,3.916667,4.086667,7.097500,8.860000,6.764000,6.626000,6.452500,6.438000,8.833333,4.973333,6.426000,3.735000,5.695000,3.0760,4.180000,4.532000,4.656000,4.1525,3.630000,4.485000,5.350000,4.390000,2.733333,3.815000,2.260000,2.585000,2.82400,3.185000,2.406667,3.737500,2.143333,2.738571,4.643333,3.553750,3.144286,4.768333,5.682500,4.926000,4.492500,5.052500,11.115000,9.090000,4.350000,5.800000,0.0000,0.000000,0.000000
3,Крупные партнеры,13.560000,10.125000,18.000000,4.290000,23.080000,31.110000,21.100000,21.210000,26.720000,4.605000,34.480000,0.000000,13.640000,14.750000,11.9200,13.510000,29.410000,39.530000,19.3350,10.043333,33.330000,8.335000,6.256667,5.863333,12.820000,3.120000,7.140000,5.55500,6.950000,2.335000,6.020000,11.980000,4.865000,5.770000,6.580000,7.460000,2.253333,0.000000,33.330000,8.570000,23.400000,7.020000,5.740000,10.220000,0.000000,3.0300,11.110000,0.000000
4,Контент,0.000000,16.666667,100.000000,56.250000,20.000000,21.430000,0.000000,14.290000,14.290000,33.330000,40.000000,0.000000,100.000000,33.330000,0.0000,0.000000,21.430000,25.000000,25.0000,40.000000,9.090000,25.000000,22.725000,75.000000,16.665000,11.906667,11.110000,9.09000,18.750000,5.260000,8.335000,7.140000,11.760000,33.336667,1.830000,10.715000,10.490000,7.140000,10.416667,14.290000,61.110000,19.230000,33.330000,44.443333,7.140000,19.4475,13.890000,10.530000
5,SEO,4.100000,4.312000,3.562000,6.332500,4.270000,4.393333,5.813333,4.148333,3.980000,3.231667,3.741667,2.708571,3.051667,7.936667,4.3940,3.058333,4.360000,4.746667,3.2720,2.251667,3.360000,4.008333,3.020000,4.247143,2.514444,4.092857,2.581250,1.69125,3.405000,3.928750,4.225000,2.494286,2.884286,3.917500,3.365714,4.240000,3.998571,3.108571,3.005556,5.366000,5.828333,4.102500,7.424000,3.928571,3.686667,3.2175,3.285714,2.161429
6,Direct,2.175000,1.647500,5.376667,3.186667,3.066667,2.150000,4.406667,2.648333,2.454000,2.397500,1.926000,1.338000,1.538000,1.997500,2.4000,1.915000,3.302500,3.432000,1.4700,1.862500,2.642000,3.460000,1.985000,1.867500,4.576667,2.805000,5.756667,2.03250,3.543333,2.786667,4.903333,2.315000,1.150000,3.230000,1.965000,3.050000,2.584000,2.245000,3.264000,3.122500,4.417500,3.052500,3.204000,3.364000,4.192000,2.4750,6.170000,4.650000
7,CPA,11.666667,4.347500,5.576667,4.500000,4.772500,3.877500,5.535000,3.885000,3.526667,3.235000,2.618000,3.077500,3.396000,3.505000,3.5875,1.515000,2.397500,3.292500,3.2280,3.655000,4.486667,3.045000,4.127500,7.410000,5.102500,4.937500,5.187500,5.90000,5.790000,5.050000,8.775000,7.730000,3.720000,3.518000,3.562000,3.606000,5.612500,3.912000,3.820000,3.205000,5.870000,3.621667,5.440000,4.852000,5.616000,6.8100,5.616000,6.590000


In [62]:
dr_vidachy = dr[dr['status_x'].isin(['APPROVED',
                                     'ISSUED'])].groupby(['Отдел',
                                                                     'week',
                                                                     'product_type'],as_index=False).agg({'dt':'count'})
dr_vidachy['Отдел'].unique()
dr_vidachy = dr_vidachy[dr_vidachy['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено','Контент','КЦ'])]
dr_vidachy_pivot = dr_vidachy.pivot_table(index=['Отдел','product_type'],
                                      columns='week',
                                      values='dt',aggfunc='sum').sort_index(axis='columns',
                                                                                level='week',
                                                                                ascending=False)
dr_vidachy_pivot=dr_vidachy_pivot.fillna(0)
dr_vidachy_pivot =dr_vidachy_pivot.sort_values('Отдел',ascending=False)

dr_vidachy_pivot = dr_vidachy_pivot.reset_index().copy()
dr_vidachy_pivot.to_excel('Недельный отчет/Vidachy_Alexander_raz.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(9) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/Vidachy_Alexander_raz.xlsx'
                  )
set_with_dataframe(worksheet, df)
dr_vidachy_pivot

week,Отдел,product_type,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2
0,Своя CPA,REFINANCE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0,3.0,0.0,1.0,0.0,2.0,4.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Своя CPA,LOAN,1.0,3.0,1.0,2.0,3.0,9.0,5.0,10.0,8.0,12.0,8.0,9.0,3.0,11.0,10.0,11.0,12.0,12.0,9.0,9.0,15.0,22.0,12.0,14.0,19.0,14.0,19.0,27.0,31.0,18.0,14.0,13.0,32.0,51.0,44.0,39.0,77.0,128.0,106.0,123.0,88.0,120.0,98.0,88.0,84.0,94.0,70.0,89.0
2,Своя CPA,CREDIT_CARD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,2.0,0.0,1.0,0.0,1.0,3.0,1.0,4.0,1.0
3,Своя CPA,CASH_LOAN,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,4.0,1.0,1.0,4.0,2.0,0.0,2.0,3.0,2.0,2.0,5.0,3.0,3.0,1.0,2.0,0.0,3.0,1.0,3.0,4.0,0.0,2.0,8.0,4.0,5.0,3.0,5.0,4.0,6.0,1.0,1.0,0.0,3.0,3.0,7.0,3.0,5.0,1.0
4,Перфоманс,LOAN,0.0,0.0,0.0,1.0,2.0,7.0,3.0,4.0,3.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,5.0,2.0,1.0,0.0,1.0,0.0,2.0,2.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Перфоманс,CREDIT_CARD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Перфоманс,CASH_LOAN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,2.0,1.0,0.0,0.0,2.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0
7,Мобильное приложение 1,CREDIT_PLEDGE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Мобильное приложение 1,AUTOCREDIT,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Мобильное приложение 1,CASH_LOAN,0.0,0.0,0.0,0.0,0.0,15.0,11.0,9.0,24.0,24.0,18.0,20.0,8.0,15.0,17.0,23.0,19.0,16.0,23.0,8.0,6.0,10.0,10.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,3.0,4.0,15.0,9.0,10.0,12.0,8.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0


In [63]:
dr_vidachy = dr[dr['status_x'].isin(['APPROVED',
                                     'ISSUED'])].groupby(['Отдел','product_type',
                                                                     'week'],as_index=False).agg({'dt':'count'})
dr_vidachy['Отдел'].unique()
dr_vidachy = dr_vidachy[dr_vidachy['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено','Контент','КЦ'])]
dr_vidachy_pivot = dr_vidachy.pivot_table(index=['Отдел'],
                                      columns='week',
                                      values='dt',aggfunc='sum').sort_index(axis='columns',
                                                                                level='week',
                                                                                ascending=False)
dr_vidachy_pivot=dr_vidachy_pivot.fillna(0)
dr_vidachy_pivot=dr_vidachy_pivot.sort_values('Отдел',ascending=False)
dr_vidachy_pivot = dr_vidachy_pivot.reset_index().copy()
dr_vidachy_pivot.to_excel('Недельный отчет/Vidachy_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(10) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/Vidachy_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
dr_vidachy_pivot

week,Отдел,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2
0,Своя CPA,1.0,3.0,1.0,2.0,3.0,10.0,8.0,10.0,10.0,13.0,13.0,10.0,4.0,15.0,12.0,12.0,14.0,17.0,13.0,12.0,23.0,25.0,17.0,15.0,23.0,18.0,24.0,30.0,37.0,24.0,16.0,15.0,41.0,57.0,49.0,44.0,84.0,132.0,114.0,126.0,89.0,121.0,101.0,92.0,94.0,98.0,79.0,91.0
1,Перфоманс,0.0,0.0,0.0,1.0,2.0,7.0,3.0,4.0,4.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,8.0,5.0,3.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,3.0,3.0,3.0,0.0,0.0,2.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Мобильное приложение 1,0.0,0.0,2.0,2.0,4.0,18.0,17.0,11.0,28.0,33.0,24.0,26.0,13.0,22.0,23.0,29.0,23.0,26.0,23.0,11.0,10.0,13.0,21.0,11.0,1.0,1.0,5.0,6.0,4.0,6.0,1.0,2.0,4.0,5.0,16.0,10.0,11.0,12.0,9.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0
3,Крупные партнеры,1.0,41.0,2.0,1.0,3.0,11.0,36.0,1.0,12.0,51.0,5.0,0.0,1.0,3.0,46.0,2.0,4.0,5.0,12.0,102.0,0.0,1.0,12.0,75.0,1.0,0.0,0.0,3.0,33.0,1.0,1.0,7.0,35.0,3.0,5.0,12.0,89.0,0.0,1.0,1.0,6.0,9.0,12.0,7.0,0.0,1.0,2.0,0.0
4,Контент,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,7.0,0.0,0.0,0.0,2.0,2.0,1.0,0.0,0.0,2.0,0.0,0.0
5,SEO,0.0,1.0,0.0,1.0,0.0,9.0,18.0,11.0,18.0,20.0,9.0,17.0,9.0,14.0,16.0,19.0,6.0,35.0,26.0,27.0,22.0,25.0,33.0,35.0,30.0,30.0,15.0,20.0,21.0,24.0,10.0,5.0,12.0,13.0,18.0,38.0,21.0,26.0,13.0,13.0,9.0,15.0,15.0,14.0,11.0,6.0,11.0,9.0
6,Direct,1.0,9.0,6.0,2.0,3.0,15.0,38.0,16.0,34.0,37.0,18.0,19.0,11.0,29.0,45.0,29.0,18.0,17.0,48.0,76.0,35.0,28.0,45.0,123.0,22.0,39.0,30.0,36.0,19.0,5.0,8.0,14.0,26.0,20.0,28.0,35.0,52.0,35.0,21.0,16.0,8.0,6.0,6.0,15.0,10.0,23.0,13.0,14.0
7,CPA,0.0,6.0,1.0,3.0,8.0,9.0,22.0,10.0,7.0,23.0,18.0,42.0,27.0,20.0,36.0,9.0,11.0,14.0,27.0,20.0,13.0,11.0,12.0,12.0,14.0,22.0,11.0,13.0,10.0,22.0,26.0,11.0,6.0,7.0,14.0,21.0,35.0,15.0,12.0,11.0,13.0,8.0,12.0,13.0,21.0,9.0,14.0,26.0


In [64]:
cr_zayavka_vidacha = dr_zayavki.merge(dr_vidachy,how='left',on=['Отдел','week','product_type'])
cr_zayavka_vidacha['cr_zayavka_vidacha_'] = ((cr_zayavka_vidacha['dt_y']/cr_zayavka_vidacha['dt_x'])*100).round(2)
cr_zayavka_vidacha_pivot = cr_zayavka_vidacha.pivot_table(index=['Отдел',
                                                                'product_type'],
                                                                 columns='week',
                                                                 values='cr_zayavka_vidacha_').sort_index(axis='columns',
                                                                                level='week',
                                                                                ascending=False)
cr_zayavka_vidacha_pivot = cr_zayavka_vidacha_pivot.fillna(0)
cr_zayavka_vidacha_pivot=cr_zayavka_vidacha_pivot.sort_values('Отдел',ascending=False)
cr_zayavka_vidacha_pivot = cr_zayavka_vidacha_pivot.reset_index().copy()
cr_zayavka_vidacha_pivot.to_excel('Недельный отчет/cr_zayavki_Vidachy_Alexander_raz.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(11) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/cr_zayavki_Vidachy_Alexander_raz.xlsx'
                  )
set_with_dataframe(worksheet, df)
cr_zayavka_vidacha_pivot

week,Отдел,product_type,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2
0,Своя CPA,REFINANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,100.00,100.00,0.00,0.00,0.00,0.00,100.00,0.00,100.00,100.00,100.00,100.00,0.00,100.00,0.00,100.00,100.00,100.00,100.00,100.00,0.00,0.00,0.00,14.29,0.00,0.00,0.00,12.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Своя CPA,LOAN,14.29,4.05,2.33,5.41,4.29,10.98,8.20,16.95,14.29,8.45,4.97,7.96,2.13,6.55,5.59,6.71,7.02,7.06,5.20,4.81,7.77,10.19,9.38,20.59,25.68,14.89,13.19,13.71,12.16,18.75,15.56,21.67,25.60,22.57,22.34,18.22,28.21,22.78,22.60,25.95,32.59,31.09,25.06,26.99,37.33,44.13,32.86,26.18
2,Своя CPA,CREDIT_CARD,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.50,0.00,0.00,0.00,0.00,11.11,0.00,0.00,0.00,6.25,0.00,5.26,8.00,7.14,0.00,0.00,6.25,0.00,2.74,0.00,0.00,6.45,5.71,0.00,2.38,0.00,3.45,6.52,2.94,13.33,5.26
3,Своя CPA,CASH_LOAN,0.00,0.00,0.00,0.00,0.00,50.00,37.50,0.00,0.00,0.00,66.67,100.00,33.33,80.00,66.67,0.00,33.33,60.00,100.00,50.00,83.33,60.00,50.00,14.29,25.00,0.00,18.75,5.56,13.64,36.36,0.00,15.38,42.11,26.67,22.73,18.75,23.81,36.36,24.00,14.29,10.00,0.00,37.50,25.00,50.00,42.86,26.32,33.33
4,Перфоманс,LOAN,0.00,0.00,0.00,14.29,25.00,50.00,42.86,57.14,42.86,66.67,0.00,0.00,0.00,33.33,33.33,0.00,50.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,75.00,41.67,18.18,5.56,0.00,50.00,0.00,100.00,40.00,0.00,100.00,20.00,66.67,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,Перфоманс,CREDIT_CARD,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,50.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,50.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,Перфоманс,CASH_LOAN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,50.00,0.00,33.33,0.00,16.67,0.00,0.00,0.00,25.00,0.00,0.00,18.18,33.33,100.00,33.33,0.00,0.00,100.00,0.00,25.00,14.29,0.00,0.00,0.00,0.00,0.00
7,Мобильное приложение 1,CREDIT_PLEDGE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,Мобильное приложение 1,AUTOCREDIT,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00,100.00,0.00,100.00,0.00,100.00,0.00,100.00,100.00,0.00,0.00,0.00,0.00,100.00,100.00,0.00,0.00,0.00,0.00,100.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,Мобильное приложение 1,CASH_LOAN,0.00,0.00,0.00,0.00,0.00,30.00,22.92,19.57,42.11,39.34,33.96,48.78,28.57,50.00,47.22,48.94,43.18,39.02,76.67,53.33,28.57,35.71,31.25,0.00,0.00,0.00,0.00,3.45,8.33,0.00,0.00,2.50,3.12,4.65,12.61,16.36,18.87,33.33,40.00,0.00,8.33,11.11,0.00,0.00,50.00,0.00,0.00,0.00


In [65]:
cr_zayavka_vidacha

,Отдел,week,product_type,dt_x,dt_y,cr_zayavka_vidacha_
0,CPA,2,CASH_LOAN,53,17.0,32.08
1,CPA,2,CREDIT_CARD,13,1.0,7.69
2,CPA,2,LOAN,35,8.0,22.86
3,CPA,2,REFINANCE,11,NaN,NaN
4,CPA,3,CASH_LOAN,110,6.0,5.45
...,...,...,...,...,...,...
1330,Своя CPA,48,CASH_LOAN,4,NaN,NaN
1331,Своя CPA,48,CREDIT_CARD,1,NaN,NaN
1332,Своя CPA,48,LOAN,74,3.0,4.05
1333,Своя CPA,49,CASH_LOAN,1,NaN,NaN


In [66]:
cr_zayavka_vidacha = dr_zayavki.merge(dr_vidachy,how='left',on=['Отдел','week','product_type'])
cr_zayavka_vidacha['cr_zayavka_vidacha_'] = ((cr_zayavka_vidacha['dt_y']/cr_zayavka_vidacha['dt_x'])*100).round(2)
cr_zayavka_vidacha_pivot = cr_zayavka_vidacha.pivot_table(index=['Отдел'
                                                            ],
                                                                 columns='week',
                                                                 values='cr_zayavka_vidacha_').sort_index(axis='columns',
                                                                                level='week',
                                                                                ascending=False)
cr_zayavka_vidacha_pivot = cr_zayavka_vidacha_pivot.fillna(0)
cr_zayavka_vidacha_pivot=cr_zayavka_vidacha_pivot.sort_values('Отдел',ascending=False)
cr_zayavka_vidacha_pivot = cr_zayavka_vidacha_pivot.reset_index().copy()
cr_zayavka_vidacha_pivot.to_excel('Недельный отчет/cr_zayavki_Vidachy_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(12) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/cr_zayavki_Vidachy_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
cr_zayavka_vidacha_pivot

week,Отдел,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2
0,Своя CPA,14.29,4.050,2.33,5.410,4.290,30.490000,22.8500,16.950000,57.145000,54.225000,57.213333,53.980000,17.730000,43.275000,36.130000,53.355000,20.175000,44.890000,68.400000,51.603333,63.700000,35.095000,42.6225,17.440000,50.226667,57.445000,34.547500,39.756667,32.765000,21.036667,11.350,18.525000,27.333333,18.496667,22.535000,13.236667,21.506667,29.5700,17.683333,15.316667,21.295000,16.735000,31.280000,18.480000,31.283333,29.976667,24.170000,21.590000
1,Перфоманс,0.00,0.000,0.00,14.290,25.000,50.000000,42.8600,57.140000,71.430000,66.670000,0.000000,0.000000,0.000000,33.330000,33.330000,0.000000,50.000000,0.000000,0.000000,0.000000,100.000000,100.000000,50.0000,0.000000,52.776667,41.670000,17.425000,5.560000,0.000000,50.000000,25.000,100.000000,40.000000,18.180000,61.110000,60.000000,50.000000,0.0000,0.000000,100.000000,0.000000,25.000000,14.290000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Мобильное приложение 1,0.00,0.000,3.57,17.705,13.815,43.870000,56.6075,46.523333,48.620000,47.413333,59.237500,45.957500,48.930000,43.565000,53.156000,60.756667,49.275000,69.622000,76.670000,56.666667,64.285000,40.242500,55.2200,59.942500,100.000000,100.000000,54.462500,43.690000,41.666667,70.833333,100.000,11.250000,8.705000,6.170000,11.860000,58.180000,12.065000,33.3300,32.500000,0.000000,8.330000,11.110000,0.000000,0.000000,50.000000,0.000000,0.000000,0.000000
3,Крупные партнеры,12.50,11.610,22.22,33.330,20.000,26.190000,18.7500,14.290000,34.290000,59.540000,50.000000,0.000000,16.670000,16.670000,56.585000,40.000000,40.000000,29.410000,21.050000,36.575000,0.000000,11.110000,22.0600,11.610000,20.000000,0.000000,0.000000,23.080000,45.165000,33.330000,20.000,24.140000,19.130000,50.000000,36.110000,40.000000,28.930000,0.0000,50.000000,33.330000,54.550000,42.860000,57.140000,36.840000,0.000000,100.000000,100.000000,0.000000
4,Контент,0.00,0.000,0.00,0.000,0.000,100.000000,0.0000,100.000000,100.000000,50.000000,66.665000,0.000000,33.330000,0.000000,0.000000,0.000000,100.000000,100.000000,100.000000,50.000000,100.000000,33.330000,41.6650,33.330000,100.000000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,66.665000,25.355000,0.0000,0.000000,0.000000,28.570000,62.500000,50.000000,0.000000,0.000000,22.220000,0.000000,0.000000
5,SEO,0.00,4.550,0.00,8.330,0.000,47.818000,48.5440,50.446000,50.535000,72.500000,40.850000,75.657500,36.960000,63.640000,65.417500,83.333333,55.637500,59.018000,56.705000,77.404000,60.683333,66.137500,59.9750,58.185000,58.546667,65.152000,53.390000,63.988000,46.684000,57.586667,44.588,37.553333,8.817500,10.880000,16.697500,20.892000,21.767500,22.0375,17.736667,33.802500,9.853333,11.360000,22.107500,12.523333,19.972500,10.876667,35.067500,18.120000
6,Direct,50.00,9.095,13.95,8.000,7.085,71.210000,77.5500,51.053333,72.360000,51.880000,38.500000,58.807500,62.306667,70.236667,36.726667,33.507500,55.286667,55.770000,83.363333,62.250000,50.693333,67.746667,27.0950,37.302500,33.630000,60.736667,18.005000,60.676667,27.895000,9.880000,10.390,16.225000,17.720000,17.735000,23.173333,25.322500,23.315000,27.3050,27.405000,22.026667,17.220000,10.240000,9.256667,39.470000,16.446667,17.276667,18.946667,19.903333
7,CPA,0.00,21.430,3.23,4.805,10.000,37.286667,35.5050,30.745000,13.846667,43.330000,52.730000,55.743333,40.120000,53.463333,57.953333,39.712500,46.570000,44.046667,65.114000,44.980000,18.630000,44.150000,37.3075,33.516667,39.593333,37.995000,43.056667,40.483333,37.550000,53.705000,30.975,10.065000,12.995000,10.970000,17.866667,16.785000,20.993333,10.7400,21.025000,38.970000,12.030000,6.183333,9.960000,7.706667,19.326667,11.405000,16.520000,20.876667


In [67]:
cr_zayavka_vidacha

,Отдел,week,product_type,dt_x,dt_y,cr_zayavka_vidacha_
0,CPA,2,CASH_LOAN,53,17.0,32.08
1,CPA,2,CREDIT_CARD,13,1.0,7.69
2,CPA,2,LOAN,35,8.0,22.86
3,CPA,2,REFINANCE,11,NaN,NaN
4,CPA,3,CASH_LOAN,110,6.0,5.45
...,...,...,...,...,...,...
1330,Своя CPA,48,CASH_LOAN,4,NaN,NaN
1331,Своя CPA,48,CREDIT_CARD,1,NaN,NaN
1332,Своя CPA,48,LOAN,74,3.0,4.05
1333,Своя CPA,49,CASH_LOAN,1,NaN,NaN


In [68]:
since = date(2021,1,4)
till=date(2021,12,31)

In [69]:
# df = get_exact_claims(since, till, client)
# df['requestId'] = df[df['partner']=='MoneyMan']['statusFields'].apply(mm_requestId)
# dp = get_kibana_postback(es, since)
# def float_apply(x):
#     try:
#         return float(x)
#     except ValueError:
#         return None
# dp['payout'] = dp['payout'].apply(float_apply)
# dp['payout'] = dp.apply(lambda x: x['payout']*1.2 if x['partnerName']=='admitad' else x['payout'], axis=1)
# dpbk = dpbk[dpbk['status'].isin(['APPROVED', 'approved', 'ISSUED'])].sort_values('payout').drop_duplicates('transitionId', keep='last')

# dr = pd.merge(left=df, right=dp, left_on='id', right_on='transitionId', how='left')
# dc = get_kibana_claims(es, since,till)[['clickId', 'userId', 'utm_campaign', 'utm_medium', 
#                                         'utm_source', 'utm_term','utm_content','gaId', 'yandexClientID', 'sourceGlobal']]
# # print(dr.columns)
# dr = pd.merge(left=dr, right=dc[dc['clickId'].notnull()].drop_duplicates('clickId'), on='clickId', how='left')
# # print(dr.columns)

# cre = cre_get(es, since)
# dr['score'] = dr['userId_x'].map(cre).fillna(0).astype(int)
# dr['whence_source'] = dr['whence_x'].apply(lambda x: x.split("_")[0] if x.find('_')>-1 and x.find('upside')<0 else x)
# # dr['payout'] = dr.apply(lambda x: 1445 if x['partner']=='MigONE' and pd.isnull(x['payout']) else x['payout'], axis=1)
# # dr['payout'] = dr.apply(lambda x: 3200 if x['partner']=='MoneyMan' and pd.isnull(x['payout']) else x['payout'], axis=1)
# # dr['payout'] = dr.apply(lambda x: 3800 if x['partner']=='Альфа-Банк' and pd.isnull(x['payout']) else x['payout'], axis=1)
# # dr['payout'] = dr.apply(lambda x: 3000 if x['partner']=='Тинькофф Банк' and pd.isnull(x['payout']) else x['payout'], axis=1)
# # dr['payout'] = dr.apply(lambda x: 3437 if x['partner']=='МигКредит' and pd.isnull(x['payout']) else x['payout'], axis=1)
# # dr['payout'] = dr.apply(lambda x: 700 if x['partner']=='CarMoney' and pd.isnull(x['payout']) else x['payout'], axis=1)

# dreg = get_registrations(dr['userId_x'].tolist())
# regs = dreg['userId'].tolist()
# dr['registration'] = dr['userId_x'].apply(lambda x: x if x in regs else np.nan)
# dreg.index = dreg['userId']
# reg_time = dreg['reg_time'].to_dict()
# dreg = dreg[dreg['utm_source'].notnull()]['utm_source'].to_dict()

# dr['Целевое действие'] = dr['productId_x'].apply(lambda x: 'Заявка' if x in claim_offers else 'Выдача')

# dr['source'] = dr.apply(lambda x: x['sourceGlobal'] if pd.notnull(x['sourceGlobal']) 
#          else (x['utm_source_y'] if pd.notnull(x['utm_source_y']) 
#          else x['whence_source']),axis=1).fillna('Direct&organic')
# dr['source_type'] = dr.apply(lambda x:'WhiteLabel' if pd.notnull(x['sourceGlobal']) 
#          else 'utm_source_y' if pd.notnull(x['utm_source_y']) 
#          else 'whence_source',axis=1).fillna('other')
# # dr.fillna('').groupby(['utm_source_y', 'sourceGlobal', 'whence_source']).count()
# dr['source'] = dr['source'].apply(lambda x: x.replace('test_partner', 'tele2')\
# .replace('tele_2', 'tele2'))#.replace('showcase', 'Direct&organic'))

# dr['source_ext'] = dr.apply(lambda x: f"{x['source']} ({dreg[x['userId_x']]})" 
#          if x['userId_x'] in dreg.keys() and  x['source'] in ['email']
#                             else x['source'],axis=1)
# dr['reg_time'] = dr['userId_x'].map(reg_time)
# dr[(dr['status_x'].isin(['APPROVED', 'ISSUED']))].groupby(['partner']).agg({'_id':'count',
#                              'payout': 'sum'}).to_excel('reports/part.xlsx')

# #dr['source'] = dr['source_ext']

# # dr['source']=dr.apply(lambda row: row['utm_source_y'] if 
# #                       pd.notnull(row['utm_source_y']) else row['whence_source'],axis=1)
# # dr['source']=dr.apply(lambda row: row['sourceGlobal'] if 
# #                       pd.notnull(row['sourceGlobal']) else row['whence_source'],axis=1)
# # dr['source']=dr.apply(lambda row: 'Город Тройка' if 
# #                       row['utm_content_y']=='gorodtroika' else row['whence_source'],axis=1)
# dr['source'] = dr['source'].apply(lambda x: 'Bankiros.ru ссылки' if 
#                                   x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
# dr['source'] = dr['source'].apply(lambda x: 'Bankiros.ru ссылки' if 
#                                   x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
# dr['source'] = dr['source'].apply(lambda x: 'Vbr.ru ссылки' if 
#                                   x in ['vbrru', 'upside_vbr','vbrr'] else x)
# dr['source'] = dr['source'].apply(lambda x: 'Email' if 
#                                  x[:5]=='email' or x[:7]=='TRIGGER' else x)
# dr['source'] = dr['source'].apply(lambda x: 'SMS рекламные' if 
#                                    x[:3]=='sms' else x)
# dr['source'] = dr['source'].apply(lambda x: 'SMS рекламные' if 
#                                    x in ['tele2TommorowWON',
#                                          'tele2Tommorow',
#                                          'tele2FourDays',
#                                          'tele2TOP'] or x[:5]=='t2sms' or x =='lostBasket' else x)
# dr['source'] = dr['source'].apply(lambda x: 'Admitad' if 
#                                    x[:7]=='admitad' else x)
# dr['source'] = dr['source'].apply(lambda x: 'Adv.Cake' if 
#                                    x[:7]=='advcake' else x)
# dr['source'] = dr['source'].apply(lambda x: 'Leadgid' if 
#                                    x[:7]=='leadgid' else x)
# dr['source'] = dr['source'].apply(lambda x: 'Rubid' if 
#                                    x[:5]=='rubid' else x)
# dr['source'] = dr['source'].apply(lambda x: 'Cpahub' if 
#                                    x[:6]=='cpahub' or x[:6]=='Cpahub' or x== 'get4click'  else x)
# dr['source'] = dr['source'].apply(lambda x: 'Перфоманс через Артикс' if 
#                                    x[:3]=='ais' else x)
# dr['source'] = dr['source'].apply(lambda x: 'Tele2 // Микрозаймы' if 
#                                    x[:4]=='tele' else x)
# dr['source'] = dr['source'].apply(lambda x: 'Теле2 \\ Кредитные карты' if 
#                                    x[:5]=='t2_kk' else x)
# dr['source'] = dr['source'].apply(lambda x: 'Виджеты' if 
#                                    x[:10]=='substitute' else x)
# dr['source'] = dr['source'].apply(lambda x: 'Браузерные пуши (web)' if 
#                                    x[:4]=='push' else x)
# dr['source'] = dr['source'].apply(lambda x: 'Exbico' if 
#                                    x[:6]=='exbico' else x)
# dr['source'] = dr['source'].apply(lambda x: 'Ссылочная масса' if 
#                                    x[:8]=='showcase' else x)
# dr['source'] = dr['source'].apply(lambda x: 'Статьи на сайте' if 
#                                    x[:12]=='content-text' else x)
# dr['source'] = dr['source'].apply(lambda x: 'МТС' if 
#                                    x[:2]=='mt' else x)
# dr['source'] = dr['source'].apply(lambda x: 'Brobank.ru' if 
#                                    x[:7]=='brobank' else x)
# dr['source'] = dr['source'].apply(lambda x: 'Тестирование гипотез' if 
#                                   x[:6]=='yandex' or x[:6]=='google' else x)
# dr['source']=dr.apply(lambda row: 'Тестирование гипотез' if 
#                       row['utm_source_y']=='yandex' or row['utm_source_y']=='google' else row['source'],axis=1)

# dr['source'] = dr.apply(lambda row: 'Металлинвест \\ потребы' if row['productId_x']=='5e9846be79dd27000141f476' 
#                         else row['source'], axis=1)
# dr['source'] = dr.apply(lambda row: 'Металлинвест \\ потребы' if row['productId_x']=='5ef4473acf22a90001010445' 
#                         else row['source'], axis=1)


# def get_kibana_reg(es, since,till):
#     gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
#     lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

#     payload = {"size": 1000,
# "query":{"bool":{"must":[{"match_all":{}},
#                          {"match_phrase":{"eventType.keyword":"REGISTRATION"}},
#                          {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
#     d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
#     sleep(0.5)
#     doc_count = d['hits']['total']
#     hits = d['hits']['hits']
#     print(doc_count, end ="; ", flush=True)
#     res = []
#     for i in range(1, int(doc_count/1000)+1):
#         scroll = d['_scroll_id']
#         d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
#         hits = hits + d['hits']['hits']
#         print(doc_count-1000*i, end ="; ", flush=True)
#         sleep(0.5)

#     dr = pd.DataFrame([hit['_source'] for hit in hits])
#     dr.params.apply(lambda x: x.keys())
#     for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
#                 'yandexClientID','currentPage',
# #                 'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
# #                   'transaction_id', 'clickId', 
#                 'sourceGlobal']:
#         dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

#     dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
#     dr['dt_reg'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
#     return dr

# regs = get_kibana_reg(es, since,till)

# dr['utm_source_x'].value_counts()

# regs['currentPage1'] = regs['currentPage'].apply(lambda x: x[:5] if pd.notnull(x) else np.nan)
# regs['currentPage2'] = regs['currentPage'].apply(lambda x: x[:9] if pd.notnull(x) else np.nan)

# dr=pd.merge(left=dr, right=regs, how='left', on='userId')

# odin = dr[dr['dt_reg'].notnull()]
# odin=odin[odin['product_type']=='LOAN']
# odin['userId'].nunique()

# dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Android_mobile' 
#                         else row['source_x'], axis=1)
# dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Huawei_mobile' 
#                         else row['source_x'], axis=1)
# dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Apple_mobile' 
#                         else row['source_x'], axis=1)

# dr['source_x'] = dr.apply(lambda row: 'Статьи на сайте' if row['currentPage1']=='/news' 
#                         else row['source_x'], axis=1)
# dr['source_x'] = dr.apply(lambda row: 'Статьи на сайте' if row['currentPage2']=='/articles' 
#                         else row['source_x'], axis=1)
# dr['source_x'] = dr.apply(lambda row: 'Органический трафик' if row['source_x']=='Ссылочная масса' 
#                         else row['source_x'], axis=1)
# dr['source_x'] = dr.apply(lambda row: 'New (2Gis)' if row['source_x']=='2gis' 
#                         else row['source_x'], axis=1)
# dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение 1' if row['source_x']=='Мобильное приложение' 
#                         else row['source_x'], axis=1)
# dr['status_x'] = dr.apply(lambda x: 'APPROVED' if
#                         pd.notnull(x['status_y']) and x['status_y'] in(['APPROVED', 'approved', 'ISSUED']) 
#                           else x['status_x'], axis=1)
# dr['week']=(pd.to_datetime(dr['dt']).dt.week)+1
# # fields = ['source_x','week']
# # # d0 = tele2_metrika(since, till, '')
# # d1 = dr.groupby((fields),as_index=False).agg({'_id': 'count'})
# # d2 = dr[(dr['status_x']!='NEW')
# #    ].groupby((fields),as_index=False).agg({'_id': 'count'})
# d3 = dr[dr['status_x'].isin(['APPROVED', 'ISSUED'])
#    ].groupby(['source_x','week','product_type'],as_index=False).agg({'_id': 'count', 'payout': 'sum'})

# # dd1 = pd.merge(d0, d1, left_index=True, right_index=True, how='right')
# # dd2 = pd.merge(d1, d2, left_index=True, right_index=True, how='left')
# # dr_group = pd.merge(dd2, d3, left_index=True, right_index=True, how='left')
# d3.sort_index(inplace = True, ascending=False)

In [70]:
df = get_exact_claims(since, till, client)
# df['requestId'] = df[df['partner']=='MoneyMan']['statusFields'].apply(mm_requestId)
dp = get_kibana_postback(es, since)
def float_apply(x):
    try:
        return float(x)
    except ValueError:
        return None
dp['payout'] = dp['payout'].apply(float_apply)
dp['payout'] = dp.apply(lambda x: x['payout']*1.2 if x['partnerName']=='admitad' else x['payout'], axis=1)
dp = dp[dp['status'].isin(['APPROVED', 'approved', 'ISSUED'])].sort_values('payout').drop_duplicates('transitionId', keep='last')
dr = pd.merge(left=df, right=dp, left_on='id', right_on='transitionId', how='left')
dc = get_kibana_claims(es, since,till)[['clickId', 'userId', 'utm_campaign', 'utm_medium', 
                                        'utm_source', 'utm_term','utm_content','gaId', 'yandexClientID', 'sourceGlobal']]
# print(dr.columns)
dr = pd.merge(left=dr, right=dc[dc['clickId'].notnull()].drop_duplicates('clickId'), on='clickId', how='left')
# print(dr.columns)

cre = cre_get(es, since)
dr['score'] = dr['userId_x'].map(cre).fillna(0).astype(int)
dr['whence_source'] = dr['whence_x'].apply(lambda x: x.split("_")[0] if x.find('_')>-1 and x.find('upside')<0 else x)
#dr['payout'] = dr.apply(lambda x: 1445 if x['partner']=='MigONE' and pd.isnull(x['payout']) else x['payout'], axis=1)
dr['payout'] = dr.apply(lambda x: 3200 if x['partner']=='MoneyMan' and pd.isnull(x['payout']) else x['payout'], axis=1)
# dr['payout'] = dr.apply(lambda x: 3800 if x['partner']=='Альфа-Банк' and pd.isnull(x['payout']) else x['payout'], axis=1)
# dr['payout'] = dr.apply(lambda x: 3000 if x['partner']=='Тинькофф Банк' and pd.isnull(x['payout']) else x['payout'], axis=1)
# dr['payout'] = dr.apply(lambda x: 3437 if x['partner']=='МигКредит' and pd.isnull(x['payout']) else x['payout'], axis=1)
# dr['payout'] = dr.apply(lambda x: 700 if x['partner']=='CarMoney' and pd.isnull(x['payout']) else x['payout'], axis=1)

dreg = get_registrations(dr['userId_x'].tolist())
regs = dreg['userId'].tolist()
dr['registration'] = dr['userId_x'].apply(lambda x: x if x in regs else np.nan)
dreg.index = dreg['userId']
reg_time = dreg['reg_time'].to_dict()
dreg = dreg[dreg['utm_source'].notnull()]['utm_source'].to_dict()

dr['Целевое действие'] = dr['productId_x'].apply(lambda x: 'Заявка' if x in claim_offers else 'Выдача')

dr['source'] = dr.apply(lambda x: x['sourceGlobal'] if pd.notnull(x['sourceGlobal']) 
         else (x['utm_source_y'] if pd.notnull(x['utm_source_y']) 
         else x['whence_source']),axis=1).fillna('Direct&organic')
dr['source_type'] = dr.apply(lambda x:'WhiteLabel' if pd.notnull(x['sourceGlobal']) 
         else 'utm_source_y' if pd.notnull(x['utm_source_y']) 
         else 'whence_source',axis=1).fillna('other')
# dr.fillna('').groupby(['utm_source_y', 'sourceGlobal', 'whence_source']).count()
dr['source'] = dr['source'].apply(lambda x: x.replace('test_partner', 'tele2')\
.replace('tele_2', 'tele2'))#.replace('showcase', 'Direct&organic'))

dr['source_ext'] = dr.apply(lambda x: f"{x['source']} ({dreg[x['userId_x']]})" 
         if x['userId_x'] in dreg.keys() and  x['source'] in ['email']
                            else x['source'],axis=1)
dr['reg_time'] = dr['userId_x'].map(reg_time)
dr[(dr['status_x'].isin(['APPROVED', 'ISSUED']))].groupby(['partner']).agg({'_id':'count',
                             'payout': 'sum'}).to_excel('reports/part.xlsx')

dr['source'] = dr['source_ext']

# dr['source']=dr.apply(lambda row: row['utm_source_y'] if 
#                       pd.notnull(row['utm_source_y']) else row['whence_source'],axis=1)
# dr['source']=dr.apply(lambda row: row['sourceGlobal'] if 
#                       pd.notnull(row['sourceGlobal']) else row['whence_source'],axis=1)
# dr['source']=dr.apply(lambda row: 'Город Тройка' if 
#                       row['utm_content_y']=='gorodtroika' else row['whence_source'],axis=1)
dr['source'] = dr['source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
dr['source'] = dr['source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
dr['source'] = dr['source'].apply(lambda x: 'Vbr.ru ссылки' if 
                                  x in ['vbrru', 'upside_vbr','vbrr'] else x)
dr['source'] = dr['source'].apply(lambda x: 'Email' if 
                                 x[:5]=='email' or x[:7]=='TRIGGER' else x)
dr['source'] = dr['source'].apply(lambda x: 'SMS рекламные' if 
                                   x[:3]=='sms' else x)
dr['source'] = dr['source'].apply(lambda x: 'SMS рекламные' if 
                                   x in ['tele2TommorowWON',
                                         'tele2Tommorow',
                                         'tele2FourDays',
                                         'tele2TOP'] or x[:5]=='t2sms' or x =='lostBasket' else x)
dr['source'] = dr['source'].apply(lambda x: 'Admitad' if 
                                   x[:7]=='admitad' else x)
dr['source'] = dr['source'].apply(lambda x: 'Adv.Cake' if 
                                   x[:7]=='advcake' else x)
dr['source'] = dr['source'].apply(lambda x: 'Leadgid' if 
                                   x[:7]=='leadgid' else x)
dr['source'] = dr['source'].apply(lambda x: 'Rubid' if 
                                   x[:5]=='rubid' else x)
dr['source'] = dr['source'].apply(lambda x: 'Cpahub' if 
                                   x[:6]=='cpahub' or x[:6]=='Cpahub' or x== 'get4click'  else x)
dr['source'] = dr['source'].apply(lambda x: 'Перфоманс через Артикс' if 
                                   x[:3]=='ais' else x)
dr['source'] = dr['source'].apply(lambda x: 'Tele2 // Микрозаймы' if 
                                   x[:4]=='tele' else x)
dr['source'] = dr['source'].apply(lambda x: 'Теле2 \\ Кредитные карты' if 
                                   x[:5]=='t2_kk' else x)
dr['source'] = dr['source'].apply(lambda x: 'Виджеты' if 
                                   x[:10]=='substitute' else x)
dr['source'] = dr['source'].apply(lambda x: 'Браузерные пуши (web)' if 
                                   x[:4]=='push' else x)
dr['source'] = dr['source'].apply(lambda x: 'Exbico' if 
                                   x[:6]=='exbico' else x)
dr['source'] = dr['source'].apply(lambda x: 'Ссылочная масса' if 
                                   x[:8]=='showcase' else x)
dr['source'] = dr['source'].apply(lambda x: 'Статьи на сайте' if 
                                   x[:12]=='content-text' else x)
dr['source'] = dr['source'].apply(lambda x: 'МТС' if 
                                   x[:2]=='mt' else x)
dr['source'] = dr['source'].apply(lambda x: 'Brobank.ru' if 
                                   x[:7]=='brobank' else x)
dr['source'] = dr['source'].apply(lambda x: 'Тестирование гипотез' if 
                                  x[:6]=='yandex' or x[:6]=='google' else x)
dr['source']=dr.apply(lambda row: 'Тестирование гипотез' if 
                      row['utm_source_y']=='yandex' or row['utm_source_y']=='google' else row['source'],axis=1)

dr['source'] = dr.apply(lambda row: 'Металлинвест \\ потребы' if row['productId_x']=='5e9846be79dd27000141f476' 
                        else row['source'], axis=1)
dr['source'] = dr.apply(lambda row: 'Металлинвест \\ потребы' if row['productId_x']=='5ef4473acf22a90001010445' 
                        else row['source'], axis=1)
dr['status_x'] = dr.apply(lambda x: 'APPROVED' if
                        pd.notnull(x['status_y']) and x['status_y'] in(['APPROVED', 'approved', 'ISSUED']) 
                          else x['status_x'], axis=1)


def get_kibana_reg(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 1000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"REGISTRATION"}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/1000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-1000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID','currentPage',
#                 'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
#                   'transaction_id', 'clickId', 
                'sourceGlobal']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['dt_reg'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

regs = get_kibana_reg(es, since,till)

dr['utm_source_x'].value_counts()

regs['currentPage1'] = regs['currentPage'].apply(lambda x: x[:5] if pd.notnull(x) else np.nan)
regs['currentPage2'] = regs['currentPage'].apply(lambda x: x[:9] if pd.notnull(x) else np.nan)

dr=pd.merge(left=dr, right=regs, how='left', on='userId')

odin = dr[dr['dt_reg'].notnull()]
odin=odin[odin['product_type']=='LOAN']
odin['userId'].nunique()

dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Android_mobile' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Huawei_mobile' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Apple_mobile' 
                        else row['source_x'], axis=1)

dr['source_x'] = dr.apply(lambda row: 'Статьи на сайте' if row['currentPage1']=='/news' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Статьи на сайте' if row['currentPage2']=='/articles' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Органический трафик' if row['source_x']=='Ссылочная масса' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'New (2Gis)' if row['source_x']=='2gis' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение 1' if row['source_x']=='Мобильное приложение' 
                        else row['source_x'], axis=1)
dr['week']=(pd.to_datetime(dr['dt']).dt.week)+1
d3 =dr[dr['status_x'].isin(['APPROVED','ISSUED'])]
# fields = ['source_x','week']
# # d0 = tele2_metrika(since, till, '')
# d1 = dr.groupby((fields),as_index=False).agg({'_id': 'count'})
# d2 = dr[(dr['status_x']!='NEW')
#    ].groupby((fields),as_index=False).agg({'_id': 'count'})
#d3 = dr[(dr['status_x'].isin(['APPROVED', 'ISSUED']))
   #].groupby(['source_x','month','product_type'],as_index=False).agg({'_id': 'count', 'payout': 'sum'})

# dd1 = pd.merge(d0, d1, left_index=True, right_index=True, how='right')
# dd2 = pd.merge(d1, d2, left_index=True, right_index=True, how='left')
# dr_group = pd.merge(dd2, d3, left_index=True, right_index=True, how='left')
#d3.sort_index(inplace = True, ascending=False)

273506


INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:0.377s]


554810; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.233s]


544810; 

...

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.296s]


1636; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.080s]


636; 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:187: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [71]:

d3['Отдел'] = d3['source_x']
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'Direct' if 
                                   x in ['Email','SMS рекламные','Браузерные пуши (web)'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП',
                                         'накрутка поведенческих факторов',
                                         'Мобильное приложение',
                                         'Новая карточка',
                                         'Новые витрины'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'CPA' if 
                                   x in ['Adv.Cake','Admitad','Leadgid','Cpahub','Rubid','Город Тройка',
                                         'Мобильное приложение',
                                         'Металлинвест \ потребы','CPA пиар','Новые продукты \\ дебетовые',
                                         'CallTat','Металлинвест \\ РКО','Промокоды','Чек Скан'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'Своя CPA' if 
                                   x in ['Bankiros.ru ссылки','Vbr.ru WL','Vbr.ru ссылки','Brobank.ru',
                                         'Металлинвест','Bankiros.ru WL','Новые партнеры WL'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'Контент' if 
                                   x in ['Яндекс.Дзен','Статьи на сайте',
                                         'Рекламные статьи (платные публикации)'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'Крупные партнеры' if 
                                   x in ['Tele2 // Микрозаймы','Теле2 \\ Кредитные карты',
                                         'Qiwi \\ Микрозайм','New (2Gis)','Мегафон','Крупный партнер (1)',
                                         'Крупный партнер (2)','Крупный партнер (3)'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'КЦ' if 
                                   x in ['Exbico','Внешние базы','Обзвон роботом','Допродажи КЦ'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'Перфоманс' if 
                                   x in ['Перфоманс','Тендерное агентство (Артикс)','МТС \\  баннер + СМС',
                                         'Пуш партнерка','Тестирование гипотез'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'Мобильное приложение 1' if 
                                   x in ['Мобильное приложение 1'] else x)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [72]:
#d3['payout']=d3['payout'].astype('int')

In [73]:
d3

,_id,type,createdAt,updatedAt,status_x,borrower,partner,product,statusFields,whence_x,adapterName,clickId,id_x,dt,productId_x,product_type,product_name,userId_x,phone,indexName_x,fallbackResponse_x,json_x,id_y,eventType_x,source_x,params_x,timestamp_x,gaId_x,userId_y,utm_campaign_x,utm_medium_x,utm_source_x,utm_term_x,productType_x,yandexClientID_x,productId_y,whence_y,transitionId,partnerName,payout,status_y,transaction_id,externalId,orderId,transactionId,time_x,date,userId,utm_campaign_y,utm_medium_y,utm_source_y,utm_term_y,utm_content,gaId_y,yandexClientID_y,sourceGlobal_x,score,whence_source,registration,Целевое действие,source_type,source_ext,reg_time,indexName_y,fallbackResponse_y,json_y,id,eventType_y,source_y,params_y,timestamp_y,gaId,utm_campaign,utm_medium,utm_source,utm_term,productType_y,yandexClientID,currentPage,sourceGlobal_y,time_y,dt_reg,currentPage1,currentPage2,week,Отдел
20,5ff23db87976dd000132a97c,LINK,2021-01-03 21:57:12.345,2021-01-03 21:57:12.345,APPROVED,"{'_id': None, 'userId': '5ff23cbee1bdad0001f5c7fb', 'fullName': 'Неволина Ирина Вячеславовна', 'email': 'infogates@yandex.ru', 'phone': '79000941564'}",Vivus,"{'_id': 5fe0587131fcc00001239248, 'name': 'Деньги на банковскую карту от Vivus', 'productType': 'CREDIT_CARD', 'productName': 'Кредитная карта'}","{'LINK': None, 'NOTE': 'OfferId not found'}",showcase,NaN,e238c4b1-ebc6-491d-979f-265dda5b29a7,5ff23db87976dd000132a97c,2021-01-04,5fe0587131fcc00001239248,CREDIT_CARD,Деньги на банковскую карту от Vivus,5ff23cbee1bdad0001f5c7fb,79000941564,NaN,False,"{""id"":""5f200191-b4ee-4acd-9f21-469464814bc5"",""eventType"":""POSTBACK"",""timestamp"":""2021-03-05T13:36:40.588Z"",""source"":""private"",""indexName"":null,""params"":{""partnerName"":""leadssu"",""transitionId"":""5ff23db87976dd000132a97c"",""payout"":""1530"",""status"":""APPROVED""},""fallbackResponse"":false}",5f200191-b4ee-4acd-9f21-469464814bc5,POSTBACK,Органический трафик,"{'payout': '1530', 'partnerName': 'leadssu', 'transitionId': '5ff23db87976dd000132a97c', 'status': 'APPROVED'}",2021-03-05T13:36:40.588Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff23db87976dd000132a97c,leadssu,1530.0,APPROVED,NaN,NaN,NaN,NaN,2021-03-05 13:36:40,2021-03-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,showcase,5ff23cbee1bdad0001f5c7fb,Выдача,whence_source,showcase,2021-01-03 21:55:01.315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,2,SEO
139,5ff2ab1253668d0001f5b4d8,LINK,2021-01-04 05:43:46.234,2021-01-04 05:43:46.234,APPROVED,"{'_id': None, 'userId': '5ff2ab12e1bdad0001f5e1ec'}",Vivus,"{'_id': 5abb6f9a21f7ea00074349d8, 'name': 'Vivus', 'productType': 'LOAN', 'productName': 'Займ'}","{'LINK': None, 'NOTE': 'OfferId not found'}",bankiros_1_1,,NaN,5ff2ab1253668d0001f5b4d8,2021-01-04,5abb6f9a21f7ea00074349d8,LOAN,Vivus,5ff2ab12e1bdad0001f5e1ec,NaN,NaN,False,"{""id"":""ba62c724-8fda-466a-98ff-de06ea124439"",""eventType"":""POSTBACK"",""timestamp"":""2021-02-15T09:50:03.728Z"",""source"":""private"",""indexName"":null,""params"":{""partnerName"":""leadssu"",""transitionId"":""5ff2ab1253668d0001f5b4d8"",""payout"":""1530"",""status"":""APPROVED""},""fallbackResponse"":false}",ba62c724-8fda-466a-98ff-de06ea124439,POSTBACK,Bankiros.ru ссылки,"{'payout': '1530', 'partnerName': 'leadssu', 'transitionId': '5ff2ab1253668d0001f5b4d8', 'status': 'APPROVED'}",2021-02-15T09:50:03.728Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff2ab1253668d0001f5b4d8,leadssu,1530.0,APPROVED,NaN,NaN,NaN,NaN,2021-02-15 09:50:03,2021-02-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,bankiros,NaN,Выдача,whence_source,bankiros,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,2,Своя CPA
269,5ff2db9aba6ffa000141598f,LINK,2021-01-04 09:10:50.928,2021-01-04 09:10:50.928,APPROVED,"{'_id': None, 'userId': '5ff2da669bad3e0001d78726', 'fullName': 'Морозов Алексей Андреевич', 'email': 'morozov7742@yandex.ru', 'phone': '79515012017'}",Экспобанк,"{'_id': 5f509227b50341000118d7c2, 'name': 'Лёгкий

In [74]:
d3_pivot = d3[d3['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено',
                                'Контент','КЦ'])].pivot_table(index=['Отдел','product_type'],
                                                              columns='week',
                                                              values='payout',aggfunc='sum').sort_index(axis='columns',
                                                                                level='week',
                                                                                ascending=False)
d3_pivot=d3_pivot.fillna(0)
#d3_pivot['payout'] = d3_pivot['payout'].astype('int')
d3_pivot=d3_pivot.sort_values('Отдел',ascending=False)

d3_pivot.to_excel('Недельный отчет/virucka_Alexander_raz.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(13) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/virucka_Alexander_raz.xlsx'
                  )
set_with_dataframe(worksheet, df)
d3_pivot

week                                      49       48       47      46       45       44       43       42       41        40       39       38       37       36        35       34       33       32       31        30       29       28       27        26       25       24       23       22       21       20       19       18       17       16       15       14        13        12        11        10       9        8        7        6        5        4        3        2 
Отдел                  product_type                                                                                                                                                                                                                                                                                                                                                                                                                                                       
Своя CPA               REFINANCE         0.0      0.0      0.0     0.0      0.0      0.0      0.0      0.0    600.0     300.0    300.0      0.0      0.0      0.0       0.0    300.0      0.0    300.0    600.0     300.0    900.0      0.0    300.0       0.0    600.0   1200.0    300.0    600.0    300.0      0.0      0.0      0.0    150.0      0.0      0.0      0.0     300.0       0.0       0.0       0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0
                       LOAN           2160.0   4680.0   2160.0  5840.0  19435.0  37300.0  12445.0  43345.0  24465.0   44100.0  24400.0  35760.0  12600.0  40950.0   25880.0  37120.0  31980.0  43440.0  38000.0   25340.0  55800.0  57630.0  26500.0   23170.0  28940.0  44070.0  27865.0  70090.0  32965.0  25580.0  20700.0  14040.0  33555.0  44182.0  50435.0  47095.0   92864.0  141929.0  100894.0  102439.0  79793.0  97274.0  65234.0  67090.0  72153.0  83470.0  74233.0  59845.0
                       CREDIT_CARD       0.0      0.0      0.0     0.0      0.0      0.0      0.0      0.0      0.0       0.0      0.0      0.0      0.0      0.0       0.0      0.0      0.0   3000.0      0.0       0.0      0.0      0.0   2300.0       0.0      0.0      0.0    810.0      0.0   3000.0   4300.0   3000.0      0.0      0.0   2000.0      0.0   4800.0       0.0       0.0    4500.0    4000.0      0.0    800.0      0.0   1530.0   6516.0   2000.0   8516.0    800.0
                       CASH_LOAN         0.0      0.0      0.0     0.0      0.0    300.0    900.0      0.0      0.0       0.0    930.0    200.0    300.0    930.0     500.0      0.0    400.0    460.0    600.0     500.0   1130.0    390.0    390.0     150.0    300.0      0.0    450.0    150.0    450.0    600.0      0.0    300.0   1200.0    600.0    730.0    410.0     710.0     600.0     820.0     140.0    150.0      0.0    430.0    430.0   1000.0    430.0    730.0    140.0
Перфоманс              LOAN              0.0      0.0      0.0  2700.0    995.0   1996.0   2595.0    790.0    565.0     150.0      0.0      0.0      0.0     75.0    1360.0      0.0    100.0      0.0      0.0       0.0      0.0      0.0      0.0       0.0   2135.0    610.0   1900.0   2100.0      0.0    297.0      0.0   2325.0   5325.0      0.0    100.0    185.0    2285.0       0.0       0.0       0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0
                       CREDIT_CARD       0.0      0.0      0.0     0.0      0.0      0.0      0.0      0.0      0.0       0.0      0.0      0.0      0.0      0.0       0.0      0.0      0.0      0.0      0.0       0.0      0.0    810.0      0.0       0.0    810.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   2000.0      0.0       0.0       0.0       0.0       0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0
                       CASH_LOAN         0.0      0.0      0.0     0.0      0.0      0.0      0.0      0.0    300.0       0.0      0.0      0.0      0.0      0.0       0.0      0.0      0.0      0.0  

In [75]:
d3_pivot = d3[d3['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено',
                                'Контент','КЦ'])].pivot_table(index='Отдел',
                                                              columns='week',
                                                              values='payout',aggfunc='sum').sort_index(axis='columns',
                                                                                level='week',
                                                                                ascending=False)
d3_pivot=d3_pivot.fillna(0)
d3_pivot=d3_pivot.sort_values('Отдел',ascending=False)
d3_pivot.to_excel('Недельный отчет/virucka_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(14) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/virucka_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
d3_pivot

week,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2
Отдел,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Своя CPA,2160.0,4680.0,2160.0,5840.0,19435.0,37600.0,13345.0,43345.0,25065.0,44400.0,25630.0,35960.0,12900.0,41880.0,26380.0,37420.0,32380.0,47200.0,39200.0,26140.0,57830.0,58020.0,29490.0,23320.0,29840.0,45270.0,29425.0,70840.0,36715.0,30480.0,23700.0,14340.0,34905.0,46782.0,51165.0,52305.0,93874.0,142529.0,106214.0,106579.0,79943.0,98074.0,65664.0,69050.0,79669.0,85900.0,83479.0,60785.0
Перфоманс,0.0,0.0,0.0,2700.0,995.0,1996.0,2595.0,790.0,865.0,150.0,0.0,0.0,0.0,75.0,1360.0,0.0,100.0,0.0,0.0,0.0,300.0,810.0,130.0,0.0,9945.0,610.0,2050.0,2100.0,0.0,297.0,150.0,2325.0,5325.0,300.0,2230.0,465.0,2415.0,0.0,0.0,280.0,0.0,290.0,140.0,0.0,0.0,0.0,0.0,0.0
Мобильное приложение 1,0.0,0.0,6450.0,800.0,10634.0,11050.0,8110.0,2775.0,15075.0,18160.0,10730.0,11260.0,6210.0,12840.0,13760.0,13100.0,10670.0,19190.0,6390.0,3310.0,2490.0,3165.0,11620.0,9809.0,300.0,300.0,1720.0,11113.0,8150.0,5010.0,300.0,1200.0,10680.0,7240.0,15970.0,1970.0,3144.0,1560.0,1350.0,0.0,140.0,140.0,0.0,0.0,280.0,0.0,0.0,0.0
Крупные партнеры,1960.0,84722.0,4500.0,800.0,5654.0,24691.0,74886.0,800.0,26890.0,128376.0,13675.0,0.0,11000.0,10220.0,116410.0,4800.0,10000.0,12740.0,12660.0,205250.0,200.0,1160.0,15755.0,131160.0,300.0,0.0,0.0,3770.0,51930.0,2100.0,3200.0,14925.0,99460.0,8825.0,13799.0,19623.0,143352.0,0.0,310.0,2100.0,7805.0,12548.0,22204.0,7944.0,0.0,300.0,285.0,0.0
Контент,0.0,0.0,0.0,0.0,0.0,300.0,0.0,300.0,300.0,300.0,430.0,0.0,130.0,0.0,0.0,0.0,300.0,200.0,300.0,300.0,130.0,130.0,2950.0,300.0,300.0,0.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,280.0,4222.0,0.0,0.0,0.0,4820.0,10230.0,210.0,0.0,0.0,1760.0,0.0,0.0
SEO,0.0,2256.0,0.0,2256.0,0.0,4937.0,8175.0,6862.0,9099.0,11570.0,3630.0,4784.0,7390.0,2720.0,9220.0,4820.0,4110.0,23134.0,11307.0,7972.0,8227.0,10774.0,15824.0,18735.0,35410.0,8910.0,8044.0,7382.0,8431.0,9937.0,3364.0,6204.4,12210.0,4437.0,9176.0,28084.0,8930.0,24044.0,6901.0,23055.0,11601.0,13090.0,10225.0,17409.0,8926.0,2870.0,14381.0,6770.0
Direct,2250.0,14930.0,10492.0,1053.0,9100.0,14252.0,10441.0,4184.0,14197.0,28728.0,18585.0,21937.0,13270.0,27310.0,25988.0,36115.0,20841.0,16620.0,20950.0,37955.0,18500.0,11484.0,55589.0,78789.0,11192.0,9132.0,18285.0,17480.0,20777.0,7405.0,16150.0,25090.0,35416.0,50055.0,22787.0,23460.0,52195.0,28809.0,10122.0,14652.0,19185.0,7424.0,11580.0,12230.0,4585.0,33629.0,11311.0,14067.0
CPA,0.0,12576.0,953.0,7890.0,16542.0,2554.0,19166.0,8210.0,10805.0,22115.0,14953.0,61057.0,12632.0,12747.0,9410.0,9145.0,4915.0,4550.0,9375.0,12874.0,13114.0,4594.0,6974.0,7752.0,4597.0,24614.0,9807.0,17802.0,2249.0,9535.0,26734.0,7192.0,4157.0,14620.0,6742.0,21719.0,31058.0,13470.0,13691.0,17573.0,33165.0,24745.0,28510.0,45832.0,33200.0,10065.0,11955.0,10000.0


In [76]:
dr.head(1)

,_id,type,createdAt,updatedAt,status_x,borrower,partner,product,statusFields,whence_x,adapterName,clickId,id_x,dt,productId_x,product_type,product_name,userId_x,phone,indexName_x,fallbackResponse_x,json_x,id_y,eventType_x,source_x,params_x,timestamp_x,gaId_x,userId_y,utm_campaign_x,utm_medium_x,utm_source_x,utm_term_x,productType_x,yandexClientID_x,productId_y,whence_y,transitionId,partnerName,payout,status_y,transaction_id,externalId,orderId,transactionId,time_x,date,userId,utm_campaign_y,utm_medium_y,utm_source_y,utm_term_y,utm_content,gaId_y,yandexClientID_y,sourceGlobal_x,score,whence_source,registration,Целевое действие,source_type,source_ext,reg_time,indexName_y,fallbackResponse_y,json_y,id,eventType_y,source_y,params_y,timestamp_y,gaId,utm_campaign,utm_medium,utm_source,utm_term,productType_y,yandexClientID,currentPage,sourceGlobal_y,time_y,dt_reg,currentPage1,currentPage2,week
0,5ff2312053668d0001f5b3b9,LINK,2021-01-03 21:03:28.953,2021-01-03 21:03:28.953,NEW,"{'_id': None, 'userId': '5ff231209bad3e0001d75cc9'}",Тинькофф Банк,"{'_id': 5dcd77050d00b700010cfb9c, 'name': 'Tinkoff Drive', 'productType': 'CREDIT_CARD', 'productName': 'Кредитная карта'}",{},vbrru,NaN,NaN,5ff2312053668d0001f5b3b9,2021-01-04,5dcd77050d00b700010cfb9c,CREDIT_CARD,Tinkoff Drive,5ff231209bad3e0001d75cc9,NaN,NaN,NaN,NaN,NaN,NaN,Vbr.ru ссылки,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,vbrru,NaN,Выдача,whence_source,vbrru,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,2


In [77]:
 #dr['Доход'] = dr.apply(lambda row: AR_count(row['score'], row['productType_x'], row['status_x']), axis=1)
# # Тут костыли для улучшения прогноза
#dr['payout'] = dr.apply(lambda row: row['payout']/3 if row['type']=='API' else row['payout'],axis=1)
# dr['Доход'] = dr['Доход'].apply(lambda x: x*3)
# dr['Доход'] = dr.apply(lambda row: row['Доход']/8 if row['source_x']=='exRejected' else row['Доход'],axis=1)
# dr['Доход'] = dr.apply(lambda row: row['Доход'] if row['partner'] in ['еКапуста','Деньги Сразу','Веб-Займ']  and row['status_x'] in ['APPROVED', 'ISSUED'] else row['Доход'],axis=1)
# dr['Доход'] = dr.apply(lambda row: row['Доход']/2 if row['source_x'] in ['Bankiros.ru ссылки']  and
#                        row['status_x'] not in ['APPROVED', 'ISSUED'] else row['Доход'],axis=1)
# dr['Доход'] = dr.apply(lambda row: row['Доход']*3 if row['source_x'] in ['Tele2 // Микрозаймы']  and
#                        row['status_x'] not in ['APPROVED', 'ISSUED'] else row['Доход'],axis=1)

In [78]:

forecast = dr[dr['status_x'].isin(['APPROVED','ISSUED'])].groupby(['week','source_x'],as_index=False).agg({"_id": 'count',
                                  "payout": 'sum'})

In [79]:
#forecast['payout'] = forecast['payout'].astype('int')
forecast

,week,source_x,_id,payout
0,2,Admitad,7,7175.0
1,2,Adv.Cake,14,2130.0
2,2,Bankiros.ru ссылки,90,59985.0
3,2,Brobank.ru,1,800.0
4,2,Cpahub,1,135.0
5,2,Email,9,2442.0
6,2,Leadgid,3,420.0
7,2,Rubid,1,140.0
8,2,SMS рекламные,4,10825.0
9,2,Браузерные пуши (web),1,800.0


In [80]:
#forecast = forecast/(date.today().day-1)*monthrange(date.today().year, date.today().month)[1]

In [81]:
#forecast = forecast.astype('int').reset_index()
forecast.columns = ['Неделя','Активность','Заявки факт', 'Выручка факт']
forecast['Расход факт'] = 0
forecast['Расход факт']=forecast.apply(lambda row: round(row['Выручка факт']*0.95)*1 if 
                       row['Активность'] in ['Adv.Cake', 'Город Тройка'] else row['Расход факт'],axis=1)
forecast['Расход факт']=forecast.apply(lambda row: round(row['Выручка факт'])*1 if 
                       row['Активность'] in ['Vbr.ru ссылки', 'Bankiros.ru ссылки'] else row['Расход факт'],axis=1)
forecast['Расход факт']=forecast.apply(lambda row: round(6000)*1 if 
                       row['Активность'] in ['Email', 'Браузерные пуши (web)'] else row['Расход факт'],axis=1)
forecast['Расход факт']=forecast.apply(lambda row: round(row['Выручка факт']*0.85)*1 if 
                       row['Активность'] in ['Brobank.ru', 'Cpahub',
                                             'Exbico', 'Leadgid', 'Admitad',
                                             'Rubid', 'Перфоманс через Артикс'] else row['Расход факт'],axis=1)
forecast['Расход факт']=forecast.apply(lambda row: round(row['Выручка факт']*1.1)*1 if 
                       row['Активность'] in ['SMS рекламные'] else row['Расход факт'],axis=1)

forecast['Расход факт']=forecast.apply(lambda row: round(row['Выручка факт']*0.75)*1 if 
                       row['Активность'] in ['Tele2 // Микрозаймы', 'mega'] else row['Расход факт'],axis=1)
forecast['Расход факт']=forecast.apply(lambda row: round(row['Выручка факт']*0.85)*1 if 
                       row['Активность'] in ['Металлинвест \\ потребы'] else row['Расход факт'],axis=1)
forecast['Расход факт'] =forecast['Расход факт'].astype('int')

In [82]:
forecast['Маржа'] = forecast['Выручка факт'] -forecast['Расход факт'] 
forecast['Отдел'] = forecast['Активность']

forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'Direct' if 
                                   x in ['Email','SMS рекламные','Браузерные пуши (web)'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП',
                                         'накрутка поведенческих факторов',
                                         'Мобильное приложение',
                                         'Новая карточка',
                                         'Новые витрины'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'CPA' if 
                                   x in ['Adv.Cake','Admitad','Leadgid','Cpahub','Rubid','Город Тройка',
                                         'Мобильное приложение',
                                         'Металлинвест \ потребы','CPA пиар','Новые продукты \\ дебетовые',
                                         'CallTat','Металлинвест \\ РКО','Промокоды','Чек Скан'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'Своя CPA' if 
                                   x in ['Bankiros.ru ссылки','Vbr.ru WL','Vbr.ru ссылки','Brobank.ru',
                                         'Металлинвест','Bankiros.ru WL','Новые партнеры WL'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'Контент' if 
                                   x in ['Яндекс.Дзен','Статьи на сайте',
                                         'Рекламные статьи (платные публикации)'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'Крупные партнеры' if 
                                   x in ['Tele2 // Микрозаймы','Теле2 \\ Кредитные карты',
                                         'Qiwi \\ Микрозайм','New (2Gis)','Мегафон','Крупный партнер (1)',
                                         'Крупный партнер (2)','Крупный партнер (3)'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'КЦ' if 
                                   x in ['Exbico','Внешние базы','Обзвон роботом','Допродажи КЦ'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'Перфоманс' if 
                                   x in ['Перфоманс','Тендерное агентство (Артикс)','МТС \\  баннер + СМС',
                                         'Пуш партнерка','Тестирование гипотез'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'Мобильное приложение 1' if 
                                   x in ['Мобильное приложение 1'] else x)
forecast = forecast[forecast['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено',
                                'Контент','КЦ'])]
forecast

,Неделя,Активность,Заявки факт,Выручка факт,Расход факт,Маржа,Отдел
0,2,Admitad,7,7175.0,6099,1076.0,CPA
1,2,Adv.Cake,14,2130.0,2024,106.0,CPA
2,2,Bankiros.ru ссылки,90,59985.0,59985,0.0,Своя CPA
3,2,Brobank.ru,1,800.0,680,120.0,Своя CPA
4,2,Cpahub,1,135.0,115,20.0,CPA
5,2,Email,9,2442.0,6000,-3558.0,Direct
6,2,Leadgid,3,420.0,357,63.0,CPA
7,2,Rubid,1,140.0,119,21.0,CPA
8,2,SMS рекламные,4,10825.0,11908,-1083.0,Direct
9,2,Браузерные пуши (web),1,800.0,6000,-5200.0,Direct


## Маркетинг

In [83]:
marketing = forecast.pivot_table(index='Отдел',columns='Неделя',values='Расход факт').sort_index(axis='columns',
                                                                                level='Неделя',
                                                                                ascending=False)
marketing=marketing.sort_values('Отдел',ascending=False)


marketing.to_excel('Недельный отчет/marketing_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(15) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/marketing_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
marketing

Неделя,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2
Отдел,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Своя CPA,2160.0,4680.0,2160.0,5840.0,19435.0,18777.5,6605.000000,43345.0,12487.5,22177.50,12723.0,17965.000000,6427.50,20870.000000,13152.5,18687.5,16160.000000,23318.0,19510.0,13010.000000,28763.00,28981.000000,14521.000000,11649.00,14852.5,22545.000000,14595.500000,35364.00,18076.500000,10030.0,11850.0,7147.50,17351.5,23196.000000,25527.500000,17174.333333,31240.666667,47479.666667,35138.666667,35319.333333,26640.333333,32651.333333,21866.666667,22918.666667,26326.333333,28512.000000,27509.666667,30332.500000
Перфоманс,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,NaN,NaN,NaN,0.000000,0.0,NaN,0.000000,NaN,NaN,NaN,0.00,0.000000,0.000000,NaN,0.0,0.000000,0.000000,0.00,NaN,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
Мобильное приложение 1,NaN,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN
Крупные партнеры,1470.0,63542.0,3375.0,600.0,4240.0,5959.0,56164.000000,600.0,20168.0,96282.00,10256.0,NaN,8250.00,7665.000000,87308.0,3600.0,7500.000000,3799.0,9495.0,74749.000000,150.00,870.000000,11816.000000,98370.00,225.0,NaN,NaN,2828.00,38948.000000,1575.0,2400.0,11194.00,74595.0,6619.000000,4124.500000,14717.000000,107514.000000,NaN,232.000000,1575.000000,5854.000000,9411.000000,16653.000000,5958.000000,NaN,225.000000,214.000000,NaN
Контент,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.00,0.0,NaN,0.00,NaN,NaN,NaN,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,0.000000,NaN,NaN
SEO,NaN,0.0,NaN,0.0,NaN,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Direct,2475.0,9134.5,3121.0,3055.0,6000.0,7439.0,7733.000000,3485.0,4508.0,9377.50,3236.5,4612.333333,3110.00,12432.500000,8984.0,7521.0,5601.500000,5271.5,7537.5,13697.500000,3220.00,3781.000000,16211.000000,22880.00,3371.5,3660.000000,3183.500000,6000.00,3334.000000,6000.0,6000.0,3591.00,9121.5,6000.000000,4068.000000,7224.000000,10213.666667,6000.000000,3313.500000,6000.000000,6877.500000,6000.000000,6509.000000,6000.000000,4106.333333,4182.500000,4328.000000,7969.333333
CPA,NaN,5439.0,905.0,3520.0,7504.0,1183.0,5933.666667,7800.0,4817.0,5067.25,2604.0,17453.666667,2883.75,3821.666667,2936.0,4244.0,1539.666667,1413.0,4187.5,3792.333333,2874.25,1330.333333,2163.666667,1677.25,1039.5,7370.333333,3045.666667,3801.75,647.333333,2043.5,5923.0,1624.25,1856.5,4218.666667,1962.333333,9611.500000,6720.250000,3857.333333,4015.666667,3938.500000,5762.600000,7231.000000,8423.333333,9819.000000,4897.500000,2878.333333,5336.000000,1742.800000


In [84]:
marzha = forecast.pivot_table(index='Отдел',columns='Неделя',values='Маржа').sort_index(axis='columns',
                                                                                level='Неделя',
                                                                                ascending=False)
marzha=marzha.sort_values('Отдел',ascending=False)

marzha=marzha.fillna(0)
marzha.to_excel('Недельный отчет/marzha_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(16) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/marzha_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
marzha

Неделя,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2
Отдел,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Своя CPA,0.0,0.0,0.0,0.0,0.0,22.5,67.5,0.0,45.0,22.5,92.0,15.000000,22.50,70.000000,37.500000,22.5,30.000000,282.000000,90.0,60.0,152.00,29.0,224.0,11.00,67.50,90.000000,117.000000,56.00,281.000000,130.00,0.0,22.50,101.0,195.000000,55.000000,260.666667,50.666667,30.000000,266.0,207.00,7.333333,40.000000,21.333333,98.0,230.000000,121.333333,316.666667,60.000000
Перфоманс,0.0,0.0,0.0,2700.0,995.0,1996.0,2595.0,790.0,865.0,150.0,0.0,0.000000,0.00,75.000000,1360.000000,0.0,100.000000,0.000000,0.0,0.0,300.00,810.0,130.0,0.00,9945.00,610.000000,2050.000000,2100.00,0.000000,297.00,150.0,2325.00,5325.0,300.000000,2230.000000,465.000000,2415.000000,0.000000,0.0,280.00,0.000000,290.000000,140.000000,0.0,0.000000,0.000000,0.000000,0.000000
Мобильное приложение 1,0.0,0.0,6450.0,800.0,10634.0,11050.0,8110.0,2775.0,15075.0,18160.0,10730.0,11260.000000,6210.00,12840.000000,13760.000000,13100.0,10670.000000,19190.000000,6390.0,3310.0,2490.00,3165.0,11620.0,9809.00,300.00,300.000000,1720.000000,11113.00,8150.000000,5010.00,300.0,1200.00,10680.0,7240.000000,15970.000000,1970.000000,3144.000000,1560.000000,1350.0,0.00,140.000000,140.000000,0.000000,0.0,280.000000,0.000000,0.000000,0.000000
Крупные партнеры,490.0,21180.0,1125.0,200.0,1414.0,6386.5,18722.0,200.0,6722.0,32094.0,3419.0,0.000000,2750.00,2555.000000,29102.000000,1200.0,2500.000000,2571.000000,3165.0,27876.0,50.00,290.0,3939.0,32790.00,75.00,0.000000,0.000000,942.00,12982.000000,525.00,800.0,3731.00,24865.0,2206.000000,2775.000000,4906.000000,35838.000000,0.000000,78.0,525.00,1951.000000,3137.000000,5551.000000,1986.0,0.000000,75.000000,71.000000,0.000000
Контент,0.0,0.0,0.0,0.0,0.0,300.0,0.0,300.0,300.0,300.0,430.0,0.000000,130.00,0.000000,0.000000,0.0,300.000000,200.000000,300.0,300.0,130.00,130.0,2950.0,300.00,300.00,0.000000,150.000000,0.00,0.000000,0.00,0.0,0.00,0.0,0.000000,0.000000,280.000000,4222.000000,0.000000,0.0,0.00,4820.000000,10230.000000,210.000000,0.0,0.000000,1760.000000,0.000000,0.000000
SEO,0.0,2256.0,0.0,2256.0,0.0,4937.0,8175.0,6862.0,9099.0,11570.0,3630.0,4784.000000,7390.00,2720.000000,9220.000000,4820.0,4110.000000,23134.000000,11307.0,7972.0,8227.00,10774.0,15824.0,18735.00,35410.00,8910.000000,8044.000000,7382.00,8431.000000,9937.00,3364.0,6204.40,12210.0,4437.000000,9176.000000,28084.000000,8930.000000,24044.000000,6901.0,23055.00,11601.000000,13090.000000,10225.000000,17409.0,8926.000000,2870.000000,14381.000000,6770.000000
Direct,-225.0,-1669.5,2125.0,-2528.5,3100.0,-313.0,-2512.5,-1393.0,2590.5,4986.5,6056.0,2700.000000,3525.00,1222.500000,4010.000000,10536.5,4819.000000,3038.500000,2937.5,5280.0,6030.00,1961.0,11583.5,16514.50,2224.50,906.000000,5959.000000,11480.00,7054.500000,1405.00,10150.0,8954.00,8586.5,44055.000000,3527.666667,4506.000000,7184.666667,8404.500000,1747.5,8652.00,2715.000000,-2288.000000,-719.000000,6230.0,-2578.000000,12632.000000,1327.500000,-3280.333333
CPA,0.0,849.0,48.0,425.0,767.0,94.0,455.0,410.0,585.5,461.5,386.6,2898.666667,274.25,427.333333,200.666667,328.5,98.666667,103.666667,500.0,499.0,404.25,201.0,161.0,260.75,109.75,834.333333,223.333333,648.75,102.333333,340.25,760.5,173.75,222.0,654.666667,285.000000,1248.000000,1044.250000,632.666667,548.0,454.75,870.400000,1017.333333,1080.000000,1639.0,635.833333,476.666667,641.500000,257.200000


In [85]:
forecast

,Неделя,Активность,Заявки факт,Выручка факт,Расход факт,Маржа,Отдел
0,2,Admitad,7,7175.0,6099,1076.0,CPA
1,2,Adv.Cake,14,2130.0,2024,106.0,CPA
2,2,Bankiros.ru ссылки,90,59985.0,59985,0.0,Своя CPA
3,2,Brobank.ru,1,800.0,680,120.0,Своя CPA
4,2,Cpahub,1,135.0,115,20.0,CPA
5,2,Email,9,2442.0,6000,-3558.0,Direct
6,2,Leadgid,3,420.0,357,63.0,CPA
7,2,Rubid,1,140.0,119,21.0,CPA
8,2,SMS рекламные,4,10825.0,11908,-1083.0,Direct
9,2,Браузерные пуши (web),1,800.0,6000,-5200.0,Direct


In [86]:
forecast['cost_income']= (((forecast['Выручка факт']/forecast['Расход факт'])*100).round(2))
forecast

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Неделя,Активность,Заявки факт,Выручка факт,Расход факт,Маржа,Отдел,cost_income
0,2,Admitad,7,7175.0,6099,1076.0,CPA,117.64
1,2,Adv.Cake,14,2130.0,2024,106.0,CPA,105.24
2,2,Bankiros.ru ссылки,90,59985.0,59985,0.0,Своя CPA,100.00
3,2,Brobank.ru,1,800.0,680,120.0,Своя CPA,117.65
4,2,Cpahub,1,135.0,115,20.0,CPA,117.39
5,2,Email,9,2442.0,6000,-3558.0,Direct,40.70
6,2,Leadgid,3,420.0,357,63.0,CPA,117.65
7,2,Rubid,1,140.0,119,21.0,CPA,117.65
8,2,SMS рекламные,4,10825.0,11908,-1083.0,Direct,90.91
9,2,Браузерные пуши (web),1,800.0,6000,-5200.0,Direct,13.33


In [87]:
forecast.replace([np.inf, -np.inf], np.nan, inplace=True)


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [88]:
forecast = forecast.fillna(0)

In [89]:
forecast

,Неделя,Активность,Заявки факт,Выручка факт,Расход факт,Маржа,Отдел,cost_income
0,2,Admitad,7,7175.0,6099,1076.0,CPA,117.64
1,2,Adv.Cake,14,2130.0,2024,106.0,CPA,105.24
2,2,Bankiros.ru ссылки,90,59985.0,59985,0.0,Своя CPA,100.00
3,2,Brobank.ru,1,800.0,680,120.0,Своя CPA,117.65
4,2,Cpahub,1,135.0,115,20.0,CPA,117.39
5,2,Email,9,2442.0,6000,-3558.0,Direct,40.70
6,2,Leadgid,3,420.0,357,63.0,CPA,117.65
7,2,Rubid,1,140.0,119,21.0,CPA,117.65
8,2,SMS рекламные,4,10825.0,11908,-1083.0,Direct,90.91
9,2,Браузерные пуши (web),1,800.0,6000,-5200.0,Direct,13.33


In [90]:
cost_income = forecast.pivot_table(index='Отдел',columns='Неделя',values='cost_income').sort_index(axis='columns',
                                                                                level='Неделя',
                                                                                ascending=False)
cost_income=cost_income.sort_values('Отдел',ascending=False)
cost_income=cost_income.fillna(0)
cost_income.to_excel('Недельный отчет/cost_income_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(17) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/cost_income_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
cost_income

Неделя,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2
Отдел,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Своя CPA,100.00,100.00,100.000,100.000,100.000,108.825,108.825000,100.00,108.825,108.825,108.795,108.825000,108.8250,108.860000,108.825000,108.825,108.825000,108.825000,108.825,108.825000,108.8050,108.735000,108.81,108.5950,108.825,108.825000,108.825,108.7750,108.815000,105.883333,50.00,108.825,108.80,108.825,108.870000,105.886667,105.906667,105.883333,105.883333,105.883333,105.730,105.883333,105.830000,105.883333,105.883333,105.873333,105.886667,108.825000
Перфоманс,0.00,0.00,0.000,0.000,0.000,0.000,0.000000,0.00,0.000,0.000,0.000,0.000000,0.0000,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,0.00,0.0000,0.000,0.000000,0.000,0.0000,0.000000,0.000000,0.00,0.000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Мобильное приложение 1,0.00,0.00,0.000,0.000,0.000,0.000,0.000000,0.00,0.000,0.000,0.000,0.000000,0.0000,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,0.00,0.0000,0.000,0.000000,0.000,0.0000,0.000000,0.000000,0.00,0.000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Крупные партнеры,133.33,133.33,133.330,133.330,133.350,66.670,133.330000,133.33,133.330,133.330,133.340,0.000000,133.3300,133.330000,133.330000,133.330,133.330000,66.660000,133.330,66.665000,133.3300,133.330000,133.34,133.3300,133.330,0.000000,0.000,133.3100,133.330000,133.330000,133.33,133.330,133.33,133.330,66.670000,133.340000,133.330000,0.000000,133.620000,133.330000,133.330,133.330000,133.330000,133.330000,0.000000,133.330000,133.180000,0.000000
Контент,0.00,0.00,0.000,0.000,0.000,0.000,0.000000,0.00,0.000,0.000,0.000,0.000000,0.0000,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,0.00,0.0000,0.000,0.000000,0.000,0.0000,0.000000,0.000000,0.00,0.000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
SEO,0.00,0.00,0.000,0.000,0.000,0.000,0.000000,0.00,0.000,0.000,0.000,0.000000,0.0000,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,0.00,0.0000,0.000,0.000000,0.000,0.0000,0.000000,0.000000,0.00,0.000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Direct,90.91,76.92,131.055,53.395,151.670,96.965,60.750000,72.98,140.920,188.230,196.745,142.896667,154.3700,130.120000,171.355000,277.915,179.715000,149.540000,151.290,199.665000,196.2900,129.320000,308.53,400.8200,133.065,111.555000,195.095,291.3300,213.520000,123.420000,269.17,245.600,247.84,834.250,155.796667,176.955000,226.130000,240.075000,125.055000,244.200000,146.580,61.865000,88.790000,203.830000,54.163333,307.780000,119.600000,48.313333
CPA,0.00,111.45,105.300,111.450,111.455,111.465,113.493333,105.26,111.455,114.685,115.174,113.520000,114.5525,113.523333,113.546667,111.450,113.523333,113.696667,111.450,113.506667,114.5525,113.546667,113.44,114.5525,114.415,113.523333,113.520,114.4975,113.506667,114.535000,114.56,114.560,111.46,113.520,113.516667,111.455000,114.685000,113.536667,113.520000,114.737500,115.084,113.370000,113.366667,114.555000,115.501667,113.526667,111.450000,115.114000


In [91]:
df_traf

,Отдел,week,visits
0,213,48,1.0
1,CPA,2,1831.0
2,CPA,3,2424.0
3,CPA,4,1736.0
4,CPA,5,1794.0
...,...,...,...
43,SEO,45,4485.0
44,SEO,46,4844.0
45,SEO,47,4387.0
46,SEO,48,4768.0


In [92]:
dr_zayavki1= dr_zayavki.groupby(['Отдел','week'],as_index=False).agg({'dt':'sum'})


In [93]:
epc = df_traf.merge(dr_zayavki1,how='left',on=['Отдел','week'])
epc['epc'] = ((epc['dt']/epc['visits'])*100).round(2)
epc=epc[epc['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено',
                                'Контент','КЦ'])].pivot_table(index='Отдел',
                                                              columns='week',
                                                              values='epc').sort_index(axis='columns',
                                                                                level='week',
                                                                                ascending=False)
epc=epc.sort_values('Отдел',ascending=False)

epc = epc.fillna(0)
epc.to_excel('Недельный отчет/epc_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(18) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/epc_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
epc

week,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2
Отдел,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Своя CPA,0.00,790.00,883.33,600.00,1366.67,790.91,1042.86,1475.00,1220.00,1480.00,5600.00,1685.71,1042.86,1186.67,994.74,1214.29,551.52,800.00,1627.27,1300.00,670.97,661.76,351.22,151.85,174.00,188.33,347.06,317.33,457.97,227.59,172.86,170.97,267.16,634.78,460.61,626.00,623.73,1175.93,729.17,614.29,241.79,439.00,365.25,382.29,518.18,1814.29,2381.82,4022.22
Перфоманс,0.00,0.35,0.00,0.97,1.10,1.31,0.77,0.48,0.46,0.40,0.06,0.07,0.00,0.30,0.26,0.51,0.84,0.29,0.37,0.11,0.28,0.58,0.40,0.18,0.21,0.38,0.58,0.55,0.28,0.71,1.96,6.13,0.72,0.65,0.72,3.48,4.47,0.50,0.58,1.56,1.26,2.21,3.05,2.02,4.08,2.33,0.61,0.05
Мобильное приложение 1,11.61,11.35,11.51,12.97,16.60,15.77,14.93,11.96,13.00,11.36,13.32,12.83,10.25,10.68,7.47,9.08,14.84,12.10,7.81,4.91,6.31,8.71,8.94,7.39,5.23,3.81,6.36,5.94,6.13,6.31,6.69,6.69,9.33,11.56,13.41,12.39,11.42,10.22,7.47,5.61,6.38,4.81,2.08,0.78,4.62,0.00,0.00,0.00
Крупные партнеры,4.06,2.76,1.71,0.48,2.37,4.00,4.28,2.22,5.54,5.22,11.63,0.00,3.11,3.85,7.21,4.67,6.49,6.51,8.54,8.04,5.71,4.33,8.29,6.14,7.58,1.54,0.91,5.76,5.15,4.39,3.21,6.55,4.73,5.94,11.30,8.29,4.91,0.00,3.77,4.84,13.41,3.34,2.90,4.33,0.00,4.17,20.00,0.00
Контент,0.00,400.00,50.00,450.00,100.00,0.00,0.00,50.00,0.00,100.00,80.00,0.00,0.00,100.00,0.00,0.00,75.00,16.67,20.00,100.00,0.00,700.00,250.00,100.00,100.00,500.00,200.00,25.00,15.79,25.00,36.36,100.00,0.00,233.33,180.00,257.14,409.09,300.00,0.00,33.33,183.33,31.25,18.18,80.00,200.00,466.67,31.25,80.00
КЦ,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.03,0.00
SEO,2.20,1.87,1.19,1.16,1.36,1.10,2.34,1.12,1.08,1.42,1.56,1.33,1.75,1.65,2.00,1.90,1.51,3.46,2.23,3.35,10.17,4.82,4.63,5.34,4.94,4.47,6.35,6.51,5.86,6.12,4.41,5.00,6.02,8.00,8.57,11.37,7.78,6.34,8.57,5.31,6.07,7.79,5.79,6.89,3.65,4.01,3.34,3.05
Direct,27.27,22.34,8.12,38.14,44.86,56.82,63.86,69.00,63.64,61.15,43.22,36.70,38.53,125.68,135.80,86.46,83.33,93.55,73.12,88.07,106.67,88.10,71.51,155.36,100.00,107.46,208.24,37.17,29.46,30.69,48.19,23.87,12.91,40.91,26.16,20.79,11.82,39.34,54.19,60.87,15.62,31.22,56.39,57.65,49.79,19.26,91.21,15.45
CPA,9.93,6.95,6.67,5.79,7.10,4.52,10.85,13.52,7.76,9.35,5.63,1.28,2.12,12.56,10.30,7.14,8.93,11.32,11.83,14.75,11.30,8.24,12.13,14.74,11.45,10.76,12.23,9.54,10.40,14.96,16.24,18.21,8.44,6.92,9.26,10.78,14.95,18.81,20.14,17.14,13.44,19.39,28.77,18.34,17.78,10.14,6.39,6.12


In [94]:
dr_zayavki1

,Отдел,week,dt
0,CPA,2,112
1,CPA,3,155
2,CPA,4,176
3,CPA,5,319
4,CPA,6,346
5,CPA,7,391
6,CPA,8,272
7,CPA,9,251
8,CPA,10,273
9,CPA,11,225


In [95]:
d3_vyr = d3[d3['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено',
                                'Контент','КЦ'])].groupby(['Отдел','week'],as_index=False).agg({'payout':'sum'})
d3_vyr                            

,Отдел,week,payout
0,CPA,2,10000.0
1,CPA,3,11955.0
2,CPA,4,10065.0
3,CPA,5,33200.0
4,CPA,6,45832.0
5,CPA,7,28510.0
6,CPA,8,24745.0
7,CPA,9,33165.0
8,CPA,10,17573.0
9,CPA,11,13691.0


In [96]:
dr_zayavki1 = dr_zayavki1.merge(d3_vyr,how='left',on=['Отдел','week'])
dr_zayavki1

,Отдел,week,dt,payout
0,CPA,2,112,10000.0
1,CPA,3,155,11955.0
2,CPA,4,176,10065.0
3,CPA,5,319,33200.0
4,CPA,6,346,45832.0
5,CPA,7,391,28510.0
6,CPA,8,272,24745.0
7,CPA,9,251,33165.0
8,CPA,10,273,17573.0
9,CPA,11,225,13691.0


In [97]:
dr_zayavki1['epl'] =(dr_zayavki1['payout']/dr_zayavki1['dt']).round(2)
dr_zayavki1

,Отдел,week,dt,payout,epl
0,CPA,2,112,10000.0,89.29
1,CPA,3,155,11955.0,77.13
2,CPA,4,176,10065.0,57.19
3,CPA,5,319,33200.0,104.08
4,CPA,6,346,45832.0,132.46
5,CPA,7,391,28510.0,72.92
6,CPA,8,272,24745.0,90.97
7,CPA,9,251,33165.0,132.13
8,CPA,10,273,17573.0,64.37
9,CPA,11,225,13691.0,60.85


In [98]:
epl = dr_zayavki1[dr_zayavki1['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено',
                                'Контент','КЦ'])].pivot_table(index='Отдел',
                                                              columns='week',
                                                              values='epl').sort_index(axis='columns',
                                                                                level='week',
                                                                                ascending=False)
epl=epl.sort_values('Отдел',ascending=False)
epl.to_excel('Недельный отчет/epl_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
worksheet = sh.get_worksheet(19) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Недельный отчет/epl_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
epl

week,49,48,47,46,45,44,43,42,41,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2
Отдел,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Своя CPA,270.0,59.24,40.75,108.15,237.01,432.18,182.81,734.66,410.90,300.00,152.56,304.75,88.36,235.28,139.58,220.12,177.91,256.52,218.99,134.05,278.03,257.87,204.79,284.39,342.99,400.62,166.24,297.65,116.19,230.91,195.87,135.28,195.00,160.21,168.31,167.11,255.09,224.46,202.31,206.55,246.74,223.40,152.35,188.15,279.54,338.19,318.62,167.91
Перфоманс,NaN,NaN,NaN,385.71,124.38,142.57,370.71,98.75,108.12,30.00,NaN,NaN,NaN,18.75,453.33,NaN,11.11,NaN,NaN,NaN,150.00,135.00,32.50,NaN,765.00,46.92,113.89,100.00,NaN,148.50,37.50,232.50,591.67,21.43,278.75,42.27,219.55,NaN,NaN,93.33,NaN,36.25,14.00,NaN,NaN,NaN,NaN,NaN
Мобильное приложение 1,NaN,NaN,55.60,7.48,82.43,94.44,73.73,28.91,106.91,132.55,83.83,105.23,65.37,142.67,172.00,135.05,107.78,210.88,114.11,94.57,65.53,58.61,184.44,245.22,10.34,12.00,28.67,188.36,173.40,125.25,6.67,19.35,72.65,49.59,75.69,17.59,39.80,31.20,39.71,NaN,7.78,10.00,NaN,NaN,35.00,NaN,NaN,NaN
Крупные партнеры,245.0,238.65,500.00,266.67,376.93,587.88,390.03,114.29,768.29,484.44,1367.50,NaN,1833.33,567.78,346.46,960.00,1000.00,749.41,218.28,467.54,20.00,105.45,271.64,303.61,60.00,NaN,NaN,269.29,387.54,420.00,640.00,514.66,534.73,1470.83,689.95,654.10,468.47,NaN,155.00,700.00,709.55,597.52,1057.33,418.11,NaN,300.00,95.00,NaN
Контент,NaN,NaN,NaN,NaN,NaN,100.00,NaN,300.00,150.00,150.00,107.50,NaN,43.33,NaN,NaN,NaN,100.00,100.00,300.00,75.00,65.00,18.57,590.00,100.00,100.00,NaN,37.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.56,93.82,NaN,NaN,NaN,438.18,2046.00,105.00,NaN,NaN,125.71,NaN,NaN
SEO,NaN,25.35,NaN,40.29,NaN,91.43,77.86,131.96,193.60,199.48,63.68,108.73,142.12,54.40,148.71,90.94,105.38,262.89,213.34,117.24,152.35,112.23,158.24,136.75,292.64,84.06,63.84,57.67,64.36,75.28,38.23,56.40,80.33,24.38,51.84,112.79,46.03,168.14,56.57,202.24,101.76,85.56,94.68,142.70,125.72,30.21,177.54,104.15
Direct,750.0,136.97,100.88,28.46,189.58,285.04,197.00,60.64,225.35,337.98,364.41,548.42,315.95,293.66,236.25,435.12,277.88,191.03,308.09,244.87,231.25,155.19,434.29,301.87,138.17,126.83,103.31,246.20,292.63,119.44,201.88,278.78,264.30,370.78,193.11,186.19,231.98,185.86,120.50,116.29,383.70,107.59,67.33,75.49,19.35,357.76,45.43,265.42
CPA,NaN,136.70,9.93,65.75,98.46,28.70,122.86,78.94,196.45,294.87,216.71,492.40,128.90,167.72,125.47,207.84,96.37,74.59,132.04,123.79,179.64,68.57,67.71,54.21,38.63,201.75,61.68,89.91,12.49,40.06,87.65,31.82,41.99,126.03,49.21,109.14,112.94,54.53,60.85,64.37,132.13,90.97,72.92,132.46,104.08,57.19,77.13,89.29


In [99]:
#!pip install google-auth-httplib2